# DEF. MODELO ML (XGBoost - BID)

## MODULES & LIBRARIES

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#conda update scikit-learn

In [3]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
#import pandas_profiling
from math import *

import sklearn
import matplotlib
import scipy
import statsmodels
import mlxtend
import category_encoders
import seaborn
import xgboost

In [5]:
# IMPORT MODULES

# Base -----------------------------------------------------------
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

# Viz ------------------------------------------------------------
import matplotlib.pyplot as plt
import matplotlib.colors as ma
import matplotlib.font_manager as fm
import seaborn as sns
import plotly
import plotnine
from plotnine import *
#import patchworklib as pw

%matplotlib inline

In [6]:
import time
from datetime import timedelta

In [7]:
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OrdinalEncoder
from category_encoders import OrdinalEncoder

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [9]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


In [10]:
import optuna

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
%matplotlib inline

In [12]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

## 1. PREPARACIÓN DE DATOS

#### - Data (BID/OFFER MODEL)

In [208]:
# CARGA DATOS
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 60SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP).csv'
file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP).csv'
df_model = pd.read_csv(file_path)

In [209]:
cat_vars = list(df_model.select_dtypes(exclude=["number"]).columns)
cat_vars

['recordtype',
 'settlementDate',
 'BMU ID',
 'BMU Type',
 'BMU Fuel Type',
 'BMU GSP Group Name',
 'BZONE',
 'BMU Party ID',
 'Trading Unit',
 'PC Flag',
 'PC Status',
 'Exempt Export Flag',
 'Base TU Flag',
 'FPN Flag']

In [210]:
num_vars = list(df_model.select_dtypes(include=["number"]).columns)
num_vars

['settlementPeriod',
 'acceptedprice',
 'acceptedvolume',
 'LOC LAT',
 'LOC LONG',
 'LOC Center LAT',
 'LOC Center LONG',
 'BZONE GENERATION',
 'BZONE DEMAND',
 'Z1',
 'Z2',
 'Z3',
 'Z4',
 'Z5',
 'Z6',
 'Z7',
 'Z8',
 'Z9',
 'Z10',
 'Z11',
 'Z12',
 'Z13',
 'Z14',
 'Z15',
 'Z16',
 'Z17',
 'Transmission Loss Factor',
 'Generation Capacity',
 'Demand Capacity',
 'Weekend',
 'Holiday',
 'Year',
 'Month',
 'Day',
 'Weekday_SIN',
 'Weekday_COS',
 'Month_SIN',
 'Month_COS',
 'Day_SIN',
 'Day_COS',
 'settPeriod_SIN',
 'settPeriod_COS',
 'marketIndexPrice',
 'marketIndexVolume',
 'activeFlag',
 'DANF_Demand',
 'DAID_spnDemand',
 'DAIG_spnGeneration',
 'DATF_Demand',
 'SystemTotalLoad_Consumption',
 'DAMargin',
 'DAImbalance',
 'GenerationForecast_Production',
 'Solar_Generation',
 'WindOffshore_Generation',
 'WindOnshore_Generation',
 'NatGas_price',
 'Coal_price',
 'BrentOil_price',
 'sp3_pavgprice',
 'sp3_meanvolume',
 'sp3_totalvolume',
 'sp4_pavgprice',
 'sp4_meanvolume',
 'sp4_totalvolume',

In [211]:
print(f'CATEGORICAL VARIABLES ({len(cat_vars)})')
print('-'*30)
i = 0

for x in cat_vars:
      i += 1
      var_values = list(df_model[x].unique())
      if len(var_values)>15:
        ex_values = str(var_values[:15]) + '...'
      else:
        ex_values = var_values

      print(f'{i}. "{x}": {len(var_values)} unique values ({ex_values})')

CATEGORICAL VARIABLES (14)
------------------------------
1. "recordtype": 2 unique values (['BID', 'OFFER'])
2. "settlementDate": 910 unique values (['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15']...)
3. "BMU ID": 483 unique values (['E_GYAR-1', 'E_SHOS-1', 'T_CDCL-1', 'T_MRWD-1', 'T_PEMB-11', 'T_SCCL-3', 'T_SHBA-1', 'T_CARR-2', 'T_DAMC-1', 'T_GRAI-6', 'T_GRAI-7', 'T_GRAI-8', 'T_MEDP-1', 'T_SEAB-1', 'T_PEMB-31']...)
4. "BMU Type": 6 unique values (['E', 'T', '2', 'V', 'M', 'C'])
5. "BMU Fuel Type": 9 unique values (['CCGT', 'PS', 'COAL', 'OTHER', 'NPSHYD', 'OCGT', 'BATTERY', 'BIOMASS', 'WIND'])
6. "BMU GSP Group Name": 14 unique values (['Eastern England', 'South Eastern England', 'East Midlands', 'Southern England', 'Southern Wales', 'Yorkshire', 'North Western England', 'South Western England', 'Northern Scotland', 'Mersey

In [212]:
df_model = df_model.drop(columns=['BMU ID', 'BMU Party ID'], axis=1)

In [213]:
# DATE COLUMN
df_model['settlementDate'] = pd.to_datetime(df_model['settlementDate'])


# Transforming date to CATEGORICAL ORDINAL
order = list(df_model['settlementDate'].unique())
order.sort()
print(order)
data = df_model['settlementDate']
df_model['settlementDate'] = pd.Categorical(data, order, ordered=True)


[numpy.datetime64('2021-01-01T00:00:00.000000000'), numpy.datetime64('2021-01-02T00:00:00.000000000'), numpy.datetime64('2021-01-03T00:00:00.000000000'), numpy.datetime64('2021-01-04T00:00:00.000000000'), numpy.datetime64('2021-01-05T00:00:00.000000000'), numpy.datetime64('2021-01-06T00:00:00.000000000'), numpy.datetime64('2021-01-07T00:00:00.000000000'), numpy.datetime64('2021-01-08T00:00:00.000000000'), numpy.datetime64('2021-01-09T00:00:00.000000000'), numpy.datetime64('2021-01-10T00:00:00.000000000'), numpy.datetime64('2021-01-11T00:00:00.000000000'), numpy.datetime64('2021-01-12T00:00:00.000000000'), numpy.datetime64('2021-01-13T00:00:00.000000000'), numpy.datetime64('2021-01-14T00:00:00.000000000'), numpy.datetime64('2021-01-15T00:00:00.000000000'), numpy.datetime64('2021-01-16T00:00:00.000000000'), numpy.datetime64('2021-01-17T00:00:00.000000000'), numpy.datetime64('2021-01-18T00:00:00.000000000'), numpy.datetime64('2021-01-19T00:00:00.000000000'), numpy.datetime64('2021-01-20T0

In [214]:
df_model['settlementDate'].unique()

[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04, 2021-01-05, ..., 2023-06-26, 2023-06-27, 2023-06-28, 2023-06-29, 2023-06-30]
Length: 910
Categories (910, datetime64[ns]): [2021-01-01 < 2021-01-02 < 2021-01-03 < 2021-01-04 ... 2023-06-27 <
                                   2023-06-28 < 2023-06-29 < 2023-06-30]

In [215]:
# CATEGORICAL ORDINAL

# Column "settlementPeriod"
order = list(df_model['settlementPeriod'].astype('int').unique())
order.sort()
print(order)
data = df_model['settlementPeriod']
df_model['settlementPeriod'] = pd.Categorical(data, order, ordered=True)

# Column "Year"
order = list(df_model['Year'].astype('int').unique())
order.sort()
print(order)
data = df_model['Year']
df_model['Year'] = pd.Categorical(data, order, ordered=True)

# Column "Month"
order = list(df_model['Month'].astype('int').unique())
order.sort()
print(order)
data = df_model['Month']
df_model['Month'] = pd.Categorical(data, order, ordered=True)

# Column "Day"
order = list(df_model['Day'].astype('int').unique())
order.sort()
print(order)
data = df_model['Day']
df_model['Day'] = pd.Categorical(data, order, ordered=True)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
[2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [216]:
# CATEGORICAL NOMINAL

# Column "recordtype"
data = df_model['recordtype']
df_model['recordtype'] = pd.Categorical(data, ordered=False)

# Column "BMU ID"
#data = df_model['BMU ID']
#df_model['BMU ID'] = pd.Categorical(data, ordered=False)

# Column "BMU Type"
data = df_model['BMU Type']
df_model['BMU Type'] = pd.Categorical(data, ordered=False)

# Column "BMU Fuel Type"
data = df_model['BMU Fuel Type']
df_model['BMU Fuel Type'] = pd.Categorical(data, ordered=False)

# Column "BMU GSP Group Name"
data = df_model['BMU GSP Group Name']
df_model['BMU GSP Group Name'] = pd.Categorical(data, ordered=False)

# Column "BZONE"
data = df_model['BZONE']
df_model['BZONE'] = pd.Categorical(data, ordered=False)

# Column "BMU Party ID"
#data = df_model['BMU Party ID']
#df_model['BMU Party ID'] = pd.Categorical(data, ordered=False)

# Column "Trading Unit"
data = df_model['Trading Unit']
df_model['Trading Unit'] = pd.Categorical(data, ordered=False)

# Column "PC Flag"
data = df_model['PC Flag']
df_model['PC Flag'] = pd.Categorical(data, ordered=False)

# Column "PC Status"
data = df_model['PC Status']
df_model['PC Status'] = pd.Categorical(data, ordered=False)

# Column "Exempt Export Flag"
data = df_model['Exempt Export Flag']
df_model['Exempt Export Flag'] = pd.Categorical(data, ordered=False)

# Column "Base TU Flag"
data = df_model['Base TU Flag']
df_model['Base TU Flag'] = pd.Categorical(data, ordered=False)

# Column "FPN Flag"
data = df_model['FPN Flag']
df_model['FPN Flag'] = pd.Categorical(data, ordered=False)


In [217]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Columns: 209 entries, recordtype to sp48_totalvolume
dtypes: category(16), float64(191), int64(2)
memory usage: 1.7 GB


In [218]:
df_model['recordtype'].value_counts(normalize=True)

BID      0.625496
OFFER    0.374504
Name: recordtype, dtype: float64

In [219]:
#drop_vars = ['settlementDate', 'settlementPeriod', 'BMU ID', 'BMU GSP Group Name', 'BMU Party ID', 'Year', 'Month', 'Day']

In [220]:
df_model.isnull().sum()

recordtype          0
settlementDate      0
settlementPeriod    0
acceptedprice       0
acceptedvolume      0
                   ..
sp47_meanvolume     0
sp47_totalvolume    0
sp48_pavgprice      0
sp48_meanvolume     0
sp48_totalvolume    0
Length: 209, dtype: int64

In [221]:
df_model = df_model.fillna(value=99999, method=None, axis=None, inplace=False, limit=None, downcast=None)

In [222]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Columns: 209 entries, recordtype to sp48_totalvolume
dtypes: category(16), float64(191), int64(2)
memory usage: 1.7 GB


In [239]:
df_model[df_model['recordtype']=='BID']['acceptedprice'].mean()

30.589454570887952

#### - Arg. LAG Variables (BID/OFFER MODEL)

In [28]:
df_model.iloc[:,68:134].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 66 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   NatGas_price      1187159 non-null  float64
 1   Coal_price        1187159 non-null  float64
 2   BrentOil_price    1187159 non-null  float64
 3   sp3_pavgprice     1187159 non-null  float64
 4   sp3_meanvolume    1187159 non-null  float64
 5   sp3_totalvolume   1187159 non-null  float64
 6   sp4_pavgprice     1187159 non-null  float64
 7   sp4_meanvolume    1187159 non-null  float64
 8   sp4_totalvolume   1187159 non-null  float64
 9   sp5_pavgprice     1187159 non-null  float64
 10  sp5_meanvolume    1187159 non-null  float64
 11  sp5_totalvolume   1187159 non-null  float64
 12  sp6_pavgprice     1187159 non-null  float64
 13  sp6_meanvolume    1187159 non-null  float64
 14  sp6_totalvolume   1187159 non-null  float64
 15  sp7_pavgprice     1187159 non-null  float64
 16  

In [29]:
df_model.iloc[:,134:210].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 75 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sp24_pavgprice    1187159 non-null  float64
 1   sp24_meanvolume   1187159 non-null  float64
 2   sp24_totalvolume  1187159 non-null  float64
 3   sp25_pavgprice    1187159 non-null  float64
 4   sp25_meanvolume   1187159 non-null  float64
 5   sp25_totalvolume  1187159 non-null  float64
 6   sp26_pavgprice    1187159 non-null  float64
 7   sp26_meanvolume   1187159 non-null  float64
 8   sp26_totalvolume  1187159 non-null  float64
 9   sp27_pavgprice    1187159 non-null  float64
 10  sp27_meanvolume   1187159 non-null  float64
 11  sp27_totalvolume  1187159 non-null  float64
 12  sp28_pavgprice    1187159 non-null  float64
 13  sp28_meanvolume   1187159 non-null  float64
 14  sp28_totalvolume  1187159 non-null  float64
 15  sp29_pavgprice    1187159 non-null  float64
 16  

In [30]:
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

In [31]:
df_aux = pd.DataFrame()

In [32]:
sp_list = list(range(3,7))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_6_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp3_6_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp3_6_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)

In [33]:
sp_list = list(range(6,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp6_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp6_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp6_12_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)

In [34]:
'''
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp3_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp3_12_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(3,13))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp3_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp3_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp3_12_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)\n"

In [35]:
'''
sp_list = list(range(12,24))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp12_24_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp12_24_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp12_24_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(12,24))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp12_24_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp12_24_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp12_24_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)\n"

In [36]:
'''
sp_list = list(range(24,36))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp24_36_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp24_36_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp24_36_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(24,36))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp24_36_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp24_36_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp24_36_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)\n"

In [37]:
'''
sp_list = list(range(36,48))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp36_48_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp36_48_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp36_48_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(36,48))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp36_48_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp36_48_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp36_48_totalvolume'] = df_model[cols_totalvolume].mean(axis=1)\n"

In [38]:
df_aux

,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,22.893185,0.000000,0.000,27.104873,0.000000,0.000
1,37.407064,-30.982417,-451.889,37.889961,-33.565774,-914.094
2,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
3,38.280000,-92.192250,-368.769,37.630000,-89.602571,-627.218
4,35.535761,-23.795667,-230.932,38.187343,-15.829571,-110.807
...,...,...,...,...,...,...
1187154,121.012684,1.780750,7.123,119.610792,2.552571,27.380
1187155,122.405000,0.000000,0.000,117.192857,0.035071,0.491
1187156,122.405000,0.000000,0.000,117.192857,0.035071,0.491
1187157,124.000000,167.500000,670.000,120.117143,91.046429,637.325


In [39]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   sp3_6_pavgprice     1187159 non-null  float64
 1   sp3_6_meanvolume    1187159 non-null  float64
 2   sp3_6_totalvolume   1187159 non-null  float64
 3   sp6_12_pavgprice    1187159 non-null  float64
 4   sp6_12_meanvolume   1187159 non-null  float64
 5   sp6_12_totalvolume  1187159 non-null  float64
dtypes: float64(6)
memory usage: 54.3 MB


In [40]:
drop_cols = [x for x in list(df_model.columns) if x.startswith('sp')==True]
drop_cols[:5]

['sp3_pavgprice',
 'sp3_meanvolume',
 'sp3_totalvolume',
 'sp4_pavgprice',
 'sp4_meanvolume']

In [41]:
df_model = df_model.drop(columns=drop_cols, axis=1)

In [42]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 71 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1187159 non-null  category
 1   settlementDate                 1187159 non-null  category
 2   settlementPeriod               1187159 non-null  category
 3   acceptedprice                  1187159 non-null  float64 
 4   acceptedvolume                 1187159 non-null  float64 
 5   BMU Type                       1187159 non-null  category
 6   BMU Fuel Type                  1187159 non-null  category
 7   BMU GSP Group Name             1187159 non-null  category
 8   LOC LAT                        1187159 non-null  float64 
 9   LOC LONG                       1187159 non-null  float64 
 10  LOC Center LAT                 1187159 non-null  float64 
 11  LOC Center LONG                1187159 non-null  float64 
 12  

In [43]:
df_model = pd.concat([df_model, df_aux], axis=1)

In [44]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 77 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1187159 non-null  category
 1   settlementDate                 1187159 non-null  category
 2   settlementPeriod               1187159 non-null  category
 3   acceptedprice                  1187159 non-null  float64 
 4   acceptedvolume                 1187159 non-null  float64 
 5   BMU Type                       1187159 non-null  category
 6   BMU Fuel Type                  1187159 non-null  category
 7   BMU GSP Group Name             1187159 non-null  category
 8   LOC LAT                        1187159 non-null  float64 
 9   LOC LONG                       1187159 non-null  float64 
 10  LOC Center LAT                 1187159 non-null  float64 
 11  LOC Center LONG                1187159 non-null  float64 
 12  

#### - Data (SP PAVG. PRICE MODEL)

In [224]:
# CARGA DATOS
file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP (pavg. price per SP+GSP)).csv'
df_modelAVG = pd.read_csv(file_path)

In [225]:
cat_vars = list(df_modelAVG.select_dtypes(exclude=["number"]).columns)
num_vars = list(df_modelAVG.select_dtypes(include=["number"]).columns)

In [226]:
print(f'CATEGORICAL VARIABLES ({len(cat_vars)})')
print('-'*30)
i = 0

for x in cat_vars:
      i += 1
      var_values = list(df_modelAVG[x].unique())
      if len(var_values)>15:
        ex_values = str(var_values[:15]) + '...'
      else:
        ex_values = var_values

      print(f'{i}. "{x}": {len(var_values)} unique values ({ex_values})')

CATEGORICAL VARIABLES (3)
------------------------------
1. "recordtype": 2 unique values (['BID', 'OFFER'])
2. "settlementDate": 910 unique values (['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15']...)
3. "BMU GSP Group Name": 14 unique values (['East Midlands', 'Eastern England', 'London', 'Merseyside and Northern Wales', 'North Eastern England', 'North Western England', 'Northern Scotland', 'South Eastern England', 'South Western England', 'Southern England', 'Southern Scotland', 'Southern Wales', 'West Midlands', 'Yorkshire'])


In [227]:
# DATE COLUMN
df_modelAVG['settlementDate'] = pd.to_datetime(df_modelAVG['settlementDate'])


# Transforming date to CATEGORICAL ORDINAL
order = list(df_modelAVG['settlementDate'].unique())
order.sort()
print(order)
data = df_modelAVG['settlementDate']
df_modelAVG['settlementDate'] = pd.Categorical(data, order, ordered=True)


[numpy.datetime64('2021-01-01T00:00:00.000000000'), numpy.datetime64('2021-01-02T00:00:00.000000000'), numpy.datetime64('2021-01-03T00:00:00.000000000'), numpy.datetime64('2021-01-04T00:00:00.000000000'), numpy.datetime64('2021-01-05T00:00:00.000000000'), numpy.datetime64('2021-01-06T00:00:00.000000000'), numpy.datetime64('2021-01-07T00:00:00.000000000'), numpy.datetime64('2021-01-08T00:00:00.000000000'), numpy.datetime64('2021-01-09T00:00:00.000000000'), numpy.datetime64('2021-01-10T00:00:00.000000000'), numpy.datetime64('2021-01-11T00:00:00.000000000'), numpy.datetime64('2021-01-12T00:00:00.000000000'), numpy.datetime64('2021-01-13T00:00:00.000000000'), numpy.datetime64('2021-01-14T00:00:00.000000000'), numpy.datetime64('2021-01-15T00:00:00.000000000'), numpy.datetime64('2021-01-16T00:00:00.000000000'), numpy.datetime64('2021-01-17T00:00:00.000000000'), numpy.datetime64('2021-01-18T00:00:00.000000000'), numpy.datetime64('2021-01-19T00:00:00.000000000'), numpy.datetime64('2021-01-20T0

In [228]:
df_modelAVG['settlementDate'].unique()

[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04, 2021-01-05, ..., 2023-06-26, 2023-06-27, 2023-06-28, 2023-06-29, 2023-06-30]
Length: 910
Categories (910, datetime64[ns]): [2021-01-01 < 2021-01-02 < 2021-01-03 < 2021-01-04 ... 2023-06-27 <
                                   2023-06-28 < 2023-06-29 < 2023-06-30]

In [229]:
# CATEGORICAL ORDINAL

# Column "settlementPeriod"
order = list(df_modelAVG['settlementPeriod'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['settlementPeriod']
df_modelAVG['settlementPeriod'] = pd.Categorical(data, order, ordered=True)

# Column "Year"
order = list(df_modelAVG['Year'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Year']
df_modelAVG['Year'] = pd.Categorical(data, order, ordered=True)

# Column "Month"
order = list(df_modelAVG['Month'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Month']
df_modelAVG['Month'] = pd.Categorical(data, order, ordered=True)

# Column "Day"
order = list(df_modelAVG['Day'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Day']
df_modelAVG['Day'] = pd.Categorical(data, order, ordered=True)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
[2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [230]:
# CATEGORICAL NOMINAL

# Column "recordtype"
data = df_modelAVG['recordtype']
df_modelAVG['recordtype'] = pd.Categorical(data, ordered=False)

# Column "BMU GSP Group Name"
data = df_modelAVG['BMU GSP Group Name']
df_modelAVG['BMU GSP Group Name'] = pd.Categorical(data, ordered=False)


In [231]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Columns: 175 entries, recordtype to sp48_totalvolume
dtypes: category(7), float64(168)
memory usage: 604.7 MB


In [232]:
df_modelAVG['recordtype'].value_counts(normalize=True)

BID      0.506824
OFFER    0.493176
Name: recordtype, dtype: float64

In [233]:
#drop_vars = ['settlementDate', 'settlementPeriod', 'BMU ID', 'BMU GSP Group Name', 'BMU Party ID', 'Year', 'Month', 'Day']

In [234]:
df_modelAVG.isnull().sum()

recordtype            0
settlementDate        0
settlementPeriod      0
BMU GSP Group Name    0
sp_pavgprice          0
                     ..
sp47_meanvolume       0
sp47_totalvolume      0
sp48_pavgprice        0
sp48_meanvolume       0
sp48_totalvolume      0
Length: 175, dtype: int64

In [235]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Columns: 175 entries, recordtype to sp48_totalvolume
dtypes: category(7), float64(168)
memory usage: 604.7 MB


In [237]:
df_modelAVG[df_modelAVG['recordtype']=='BID']['sp_pavgprice'].mean()

72.39545051017474

#### - Arg. LAG Variables (SP PAVG. PRICE MODEL)

In [57]:
df_modelAVG.iloc[:,36:102].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 66 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   BrentOil_price    468991 non-null  float64
 1   sp3_pavgprice     468991 non-null  float64
 2   sp3_meanvolume    468991 non-null  float64
 3   sp3_totalvolume   468991 non-null  float64
 4   sp4_pavgprice     468991 non-null  float64
 5   sp4_meanvolume    468991 non-null  float64
 6   sp4_totalvolume   468991 non-null  float64
 7   sp5_pavgprice     468991 non-null  float64
 8   sp5_meanvolume    468991 non-null  float64
 9   sp5_totalvolume   468991 non-null  float64
 10  sp6_pavgprice     468991 non-null  float64
 11  sp6_meanvolume    468991 non-null  float64
 12  sp6_totalvolume   468991 non-null  float64
 13  sp7_pavgprice     468991 non-null  float64
 14  sp7_meanvolume    468991 non-null  float64
 15  sp7_totalvolume   468991 non-null  float64
 16  sp8_pavgprice     46

In [58]:
df_modelAVG.iloc[:,102:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 73 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sp24_totalvolume  468991 non-null  float64
 1   sp25_pavgprice    468991 non-null  float64
 2   sp25_meanvolume   468991 non-null  float64
 3   sp25_totalvolume  468991 non-null  float64
 4   sp26_pavgprice    468991 non-null  float64
 5   sp26_meanvolume   468991 non-null  float64
 6   sp26_totalvolume  468991 non-null  float64
 7   sp27_pavgprice    468991 non-null  float64
 8   sp27_meanvolume   468991 non-null  float64
 9   sp27_totalvolume  468991 non-null  float64
 10  sp28_pavgprice    468991 non-null  float64
 11  sp28_meanvolume   468991 non-null  float64
 12  sp28_totalvolume  468991 non-null  float64
 13  sp29_pavgprice    468991 non-null  float64
 14  sp29_meanvolume   468991 non-null  float64
 15  sp29_totalvolume  468991 non-null  float64
 16  sp30_pavgprice    46

In [59]:
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

In [60]:
df_aux = pd.DataFrame()

In [61]:
sp_list = list(range(3,7))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_6_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp3_6_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp3_6_totalvolume'] = df_modelAVG[cols_totalvolume].sum(axis=1)

In [62]:
sp_list = list(range(6,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp6_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp6_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp6_12_totalvolume'] = df_modelAVG[cols_totalvolume].sum(axis=1)

In [63]:
'''
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp3_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp3_12_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(3,13))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp3_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp3_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp3_12_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [64]:
'''
sp_list = list(range(12,24))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp12_24_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp12_24_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp12_24_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(12,24))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp12_24_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp12_24_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp12_24_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [65]:
'''
sp_list = list(range(24,36))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp24_36_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp24_36_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp24_36_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(24,36))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp24_36_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp24_36_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp24_36_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [66]:
'''
sp_list = list(range(36,48))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp36_48_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp36_48_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp36_48_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(36,48))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp36_48_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp36_48_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp36_48_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [67]:
df_aux

,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
1,73.322564,0.002250,0.009,93.948627,0.001286,0.009
2,77.072564,16.968750,67.875,88.109188,0.001286,0.009
3,73.728120,51.656250,206.625,84.133499,0.001286,0.009
4,71.737500,132.302250,529.209,80.957125,0.001286,0.009
...,...,...,...,...,...,...
468986,62.010000,-128.952750,-515.811,54.640423,-85.697333,-817.925
468987,116.460000,0.000000,0.000,118.610792,0.000000,0.000
468988,62.010000,-164.104250,-656.417,61.124296,-102.506905,-926.759
468989,62.005671,-150.664125,-675.396,61.920697,-113.622667,-924.752


In [68]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   sp3_6_pavgprice     468991 non-null  float64
 1   sp3_6_meanvolume    468991 non-null  float64
 2   sp3_6_totalvolume   468991 non-null  float64
 3   sp6_12_pavgprice    468991 non-null  float64
 4   sp6_12_meanvolume   468991 non-null  float64
 5   sp6_12_totalvolume  468991 non-null  float64
dtypes: float64(6)
memory usage: 21.5 MB


In [69]:
drop_cols = [x for x in list(df_modelAVG.columns) if x.startswith('sp')==True]
drop_cols.remove('sp_pavgprice')
drop_cols[:5]

['sp3_pavgprice',
 'sp3_meanvolume',
 'sp3_totalvolume',
 'sp4_pavgprice',
 'sp4_meanvolume']

In [70]:
df_modelAVG = df_modelAVG.drop(columns=drop_cols, axis=1)

In [71]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

In [72]:
df_modelAVG = pd.concat([df_modelAVG, df_aux], axis=1)

In [73]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

## 2. DEF. DATASETS "TRAIN" Y "TEST"

#### - Outliers (BID/OFFER MODEL)

In [74]:
#file_path1 = '/content/drive/MyDrive/QEnergy/Data/BIDS (outliers indexes SP).csv'
#file_path2 = '/content/drive/MyDrive/QEnergy/Data/OFFERS (outliers indexes SP).csv'

In [75]:
file_path1 = '/content/drive/MyDrive/QEnergy/Data/Outliers indexes SP & GSP (BID).csv'
file_path2 = '/content/drive/MyDrive/QEnergy/Data/Outliers indexes SP & GSP (OFFER).csv'

In [76]:
df_outBid = pd.read_csv(file_path1)
df_outOff = pd.read_csv(file_path2)

In [77]:
list_outBid = list(df_outBid.iloc[:,0])
list_outOff = list(df_outOff.iloc[:,0])

In [78]:
out_indlist = list_outBid.copy()
out_indlist.extend(list_outOff)

In [79]:
out_indlist[:5]

[369976, 369977, 416547, 416592, 453537]

In [80]:
df_model.shape

(1187159, 77)

In [81]:
df_model = df_model.drop(index=out_indlist, axis=0)

In [82]:
df_model.shape

(1186559, 77)

#### - ORDEN DEL DATASET INICIAL (BID/OFFER MODEL)

In [83]:
df_model = df_model.sort_values(by=['settlementDate', 'settlementPeriod'], ascending=True)
df_model = df_model.reset_index(drop=True)

In [84]:
df_model[df_model['settlementDate']=='2021-01-01'].loc[:50,]

,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,BID,2021-01-01,1,37.00,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3300.832,19.125,80.5,51.8,22.893185,0.000000,0.000,27.104873,0.000000,0.000
1,BID,2021-01-01,1,35.00,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,3300.832,19.125,80.5,51.8,37.407064,-30.982417,-451.889,37.889961,-33.565774,-914.094
2,BID,2021-01-01,1,40.00,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,3300.832,19.125,80.5,51.8,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
3,BID,2021-01-01,1,37.10,-174.768,T,CCGT,Southern England,50.898831,-1.437187,...,3300.832,19.125,80.5,51.8,38.280000,-92.192250,-368.769,37.630000,-89.602571,-627.218
4,BID,2021-01-01,1,37.00,-33.634,T,CCGT,Southern Wales,51.683003,-4.994865,...,3300.832,19.125,80.5,51.8,35.535761,-23.795667,-230.932,38.187343,-15.829571,-110.807
5,BID,2021-01-01,1,6.00,-49.000,T,CCGT,Yorkshire,53.735187,-0.243281,...,3300.832,19.125,80.5,51.8,22.893185,-62.276250,-747.315,27.104873,-47.949614,-1029.659
6,BID,2021-01-01,1,0.00,-299.000,T,CCGT,Yorkshire,53.602454,-0.144829,...,3300.832,19.125,80.5,51.8,22.893185,-62.276250,-747.315,27.104873,-47.949614,-1029.659
7,OFFER,2021-01-01,1,74.90,248.334,T,CCGT,North Western England,53.436959,-2.408212,...,3300.832,19.125,80.5,51.8,74.900000,95.864625,766.917,82.905750,64.704214,905.859
8,OFFER,2021-01-01,1,71.90,322.584,T,CCGT,East Midlands,53.307421,-0.786058,...,3300.832,19.125,80.5,51.8,73.322564,0.002250,0.009,93.948627,0.001286,0.009
9,OFFER,2021-01-01,1,72.00,270.000,T,CCGT,South Eastern England,51.425650,0.600555,...,3300.832,19.125,80.5,51.8,73.096574,125.464062,702.425,95.183547,13.761857,96.333


#### - ORDEN DEL DATASET INICIAL (SP PAVG. PRICE MODEL)

In [85]:
df_modelAVG = df_modelAVG.sort_values(by=['BMU GSP Group Name','settlementDate','settlementPeriod', 'recordtype'], ascending=True)
df_modelAVG = df_modelAVG.reset_index(drop=True)

In [86]:
df_modelAVG[df_modelAVG['settlementDate']=='2021-01-01'].loc[:50,]

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,LOC Center LAT,LOC Center LONG,Weekend,Holiday,Year,...,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,BID,2021-01-01,1,East Midlands,40.000000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
1,OFFER,2021-01-01,1,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,73.322564,0.002250,0.009,93.948627,0.001286,0.009
2,OFFER,2021-01-01,2,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,77.072564,16.968750,67.875,88.109188,0.001286,0.009
3,OFFER,2021-01-01,3,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3282.530,19.125,80.5,51.8,73.728120,51.656250,206.625,84.133499,0.001286,0.009
4,OFFER,2021-01-01,4,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3282.530,19.125,80.5,51.8,71.737500,132.302250,529.209,80.957125,0.001286,0.009
5,OFFER,2021-01-01,5,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3247.227,19.125,80.5,51.8,71.425000,218.927250,875.709,77.322231,9.697714,67.884
6,OFFER,2021-01-01,6,East Midlands,71.850000,52.800048,-0.927751,0.0,1.0,2021,...,3247.227,19.125,80.5,51.8,71.900000,269.042000,1076.168,74.392259,29.519143,206.634
7,BID,2021-01-01,20,East Midlands,24.123334,52.800048,-0.927751,0.0,1.0,2021,...,2779.294,19.125,80.5,51.8,29.500000,0.000000,0.000,16.442772,0.000000,0.000
8,BID,2021-01-01,21,East Midlands,24.581791,52.800048,-0.927751,0.0,1.0,2021,...,2626.488,19.125,80.5,51.8,29.500000,0.000000,0.000,15.299915,0.000000,0.000
9,OFFER,2021-01-01,21,East Midlands,35.000000,52.800048,-0.927751,0.0,1.0,2021,...,2626.488,19.125,80.5,51.8,72.712500,0.000000,0.000,74.464286,0.000000,0.000


#### - LABEL & ORDINAL ENCODING (BID/OFFER MODEL)

In [87]:
df_model_label = df_model.copy()

In [88]:
cat_vars = df_model.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod', 'BMU Type',
       'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
       'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag',
       'FPN Flag', 'Year', 'Month', 'Day'],
      dtype='object')

In [89]:
# Separating between categorical nominal and categorical ordinal

LabelEncoder_vars = ['recordtype', 'BMU Type', 'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
                     'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag', 'FPN Flag']

OrdinalEncoder_vars = ['settlementDate', 'settlementPeriod', 'Year', 'Month', 'Day']


In [90]:
for x in LabelEncoder_vars:

  le = LabelEncoder()

  new_var = x + '_label'
  df_model_label[new_var] = le.fit_transform(df_model_label[x])

In [91]:
for x in OrdinalEncoder_vars:

  order = list(df_model_label[x].unique())
  order.sort()

  aux_dict = {}
  for y in order:
    aux_dict[y] = order.index(y)

  maplist = [{'col':x, 'mapping':aux_dict}]
  oe = OrdinalEncoder(mapping=maplist)

  new_var = x + '_ordin'
  df_model_label[new_var] = oe.fit_transform(df_model_label[x])

In [92]:
df_model_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186559 entries, 0 to 1186558
Data columns (total 93 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1186559 non-null  category
 1   settlementDate                 1186559 non-null  category
 2   settlementPeriod               1186559 non-null  category
 3   acceptedprice                  1186559 non-null  float64 
 4   acceptedvolume                 1186559 non-null  float64 
 5   BMU Type                       1186559 non-null  category
 6   BMU Fuel Type                  1186559 non-null  category
 7   BMU GSP Group Name             1186559 non-null  category
 8   LOC LAT                        1186559 non-null  float64 
 9   LOC LONG                       1186559 non-null  float64 
 10  LOC Center LAT                 1186559 non-null  float64 
 11  LOC Center LONG                1186559 non-null  float64 
 12  

#### - LABEL & ORDINAL ENCODING (BID/OFFER MODEL)

In [93]:
df_modelAVG_label = df_modelAVG.copy()

In [94]:
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod',
       'BMU GSP Group Name', 'Year', 'Month', 'Day'],
      dtype='object')

In [95]:
# Separating between categorical nominal and categorical ordinal

LabelEncoder_vars = ['recordtype', 'BMU GSP Group Name']

OrdinalEncoder_vars = ['settlementDate', 'settlementPeriod', 'Year', 'Month', 'Day']


In [96]:
for x in LabelEncoder_vars:

  le = LabelEncoder()

  new_var = x + '_label'
  df_modelAVG_label[new_var] = le.fit_transform(df_modelAVG_label[x])

In [97]:
for x in OrdinalEncoder_vars:

  order = list(df_modelAVG_label[x].unique())
  order.sort()

  aux_dict = {}
  for y in order:
    aux_dict[y] = order.index(y)

  maplist = [{'col':x, 'mapping':aux_dict}]
  oe = OrdinalEncoder(mapping=maplist)

  new_var = x + '_ordin'
  df_modelAVG_label[new_var] = oe.fit_transform(df_modelAVG_label[x])

In [98]:
df_modelAVG_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

## 2b. PREVIOUS WORKFLOW

#### - LABEL ENCODING (SP PAVG. PRICE MODEL)

In [ ]:
df_modelAVG_label = df_modelAVG.copy()

In [ ]:
cat_vars = df_modelAVG.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod',
       'BMU GSP Group Name', 'Year', 'Month', 'Day'],
      dtype='object')

In [ ]:
le = LabelEncoder()

for x in cat_vars:

  new_var = x + '_label'
  df_modelAVG_label[new_var] = le.fit_transform(df_modelAVG_label[x])


In [ ]:
df_modelAVG_label

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,marketIndexPrice,marketIndexVolume,LOC Center LAT,LOC Center LONG,Weekend,...,sp48_pavgprice,sp48_meanvolume,sp48_totalvolume,recordtype_label,settlementDate_label,settlementPeriod_label,BMU GSP Group Name_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,East Midlands,40.000000,68.54,407.30,52.800048,-0.927751,0.0,...,36.221377,0.000000,0.000,0,0,0,0,0,0,0
1,OFFER,2021-01-01,1,East Midlands,71.900000,68.54,407.30,52.800048,-0.927751,0.0,...,69.149419,77.458000,154.916,1,0,0,0,0,0,0
2,OFFER,2021-01-01,2,East Midlands,71.900000,69.82,601.40,52.800048,-0.927751,0.0,...,67.989355,124.465000,248.930,1,0,1,0,0,0,0
3,OFFER,2021-01-01,3,East Midlands,71.900000,67.42,608.00,52.800048,-0.927751,0.0,...,67.750000,146.866000,146.866,1,0,2,0,0,0,0
4,OFFER,2021-01-01,4,East Midlands,71.900000,65.45,579.35,52.800048,-0.927751,0.0,...,67.865337,63.564000,127.128,1,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471661,BID,2023-06-30,46,Yorkshire,4.103535,85.35,2492.25,53.670497,-1.071275,0.0,...,62.792642,-18.524667,-55.574,0,909,45,13,2,5,29
471662,OFFER,2023-06-30,46,Yorkshire,169.000000,85.35,2492.25,53.670497,-1.071275,0.0,...,135.000000,2.050000,2.050,1,909,45,13,2,5,29
471663,BID,2023-06-30,47,Yorkshire,-3.207901,53.20,1380.95,53.670497,-1.071275,0.0,...,62.443450,-50.716000,-202.864,0,909,46,13,2,5,29
471664,BID,2023-06-30,48,Yorkshire,-47.158350,52.02,1497.55,53.670497,-1.071275,0.0,...,62.177127,-33.735667,-101.207,0,909,47,13,2,5,29


#### - LABEL ENCODING (BID/OFFER MODEL)

In [ ]:
df_model_label = df_model.copy()

In [ ]:
cat_vars = df_model.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod', 'BMU Type',
       'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
       'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag',
       'FPN Flag', 'Year', 'Month', 'Day'],
      dtype='object')

In [ ]:
le = LabelEncoder()

for x in cat_vars:

  new_var = x + '_label'
  df_model_label[new_var] = le.fit_transform(df_model_label[x])


In [ ]:
df_model_label

,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,37.00,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3,29,1,1,0,0,1,0,0,0
1,BID,2021-01-01,1,35.00,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,0,0,0
2,OFFER,2021-01-01,1,74.90,248.334,T,CCGT,North Western England,53.436959,-2.408212,...,16,29,1,1,0,0,1,0,0,0
3,BID,2021-01-01,1,40.00,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
4,OFFER,2021-01-01,1,71.90,322.584,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192752,BID,2023-06-30,48,-79.33,-81.150,T,WIND,East Midlands,53.480000,0.840000,...,1,29,1,1,0,0,1,2,5,29
1192753,BID,2023-06-30,48,-75.00,-2.261,T,WIND,Southern Scotland,55.347448,-3.880540,...,13,29,1,1,0,0,1,2,5,29
1192754,BID,2023-06-30,48,-75.00,-20.917,T,WIND,Southern Scotland,55.274700,-4.178600,...,13,29,1,1,0,0,1,2,5,29
1192755,BID,2023-06-30,48,-80.00,-1.875,T,WIND,Southern Scotland,55.245642,-4.307956,...,13,29,1,1,0,0,1,2,5,29


#### - SEPARACIÓN DATOS TRAIN Y TEST (BID/OFFER MODEL)

In [ ]:
df_model_label['settlementDate'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1192757 entries, 0 to 1192756
Series name: settlementDate
Non-Null Count    Dtype   
--------------    -----   
1192757 non-null  category
dtypes: category(1)
memory usage: 2.3 MB


In [ ]:
DataTrain_df = df_model_label[df_model_label['settlementDate']<'2023-06-30'].copy()
#DataTrain_df.drop(axis=1, columns=drop_vars, level=None, inplace=True, errors='raise')

In [ ]:
print(DataTrain_df.shape)
DataTrain_df.info()

(1191636, 92)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1191636 entries, 0 to 1191635
Data columns (total 92 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1191636 non-null  category
 1   settlementDate                 1191636 non-null  category
 2   settlementPeriod               1191636 non-null  category
 3   acceptedprice                  1191636 non-null  float64 
 4   acceptedvolume                 1191636 non-null  float64 
 5   marketIndexPrice               1191636 non-null  float64 
 6   marketIndexVolume              1191636 non-null  float64 
 7   BMU Type                       1191636 non-null  category
 8   BMU Fuel Type                  1191636 non-null  category
 9   BMU GSP Group Name             1191636 non-null  category
 10  LOC LAT                        1191636 non-null  float64 
 11  LOC LONG                       1191636 non-null  

In [ ]:
DataTest_df = df_model_label[df_model_label['settlementDate']>='2023-06-30'].copy()
#DataTest_df.drop(axis=1, columns=drop_vars, level=None, inplace=True, errors='raise')

In [ ]:
print(DataTest_df.shape)
DataTest_df.info()

(1121, 92)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 1191636 to 1192756
Data columns (total 92 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   recordtype                     1121 non-null   category
 1   settlementDate                 1121 non-null   category
 2   settlementPeriod               1121 non-null   category
 3   acceptedprice                  1121 non-null   float64 
 4   acceptedvolume                 1121 non-null   float64 
 5   marketIndexPrice               1121 non-null   float64 
 6   marketIndexVolume              1121 non-null   float64 
 7   BMU Type                       1121 non-null   category
 8   BMU Fuel Type                  1121 non-null   category
 9   BMU GSP Group Name             1121 non-null   category
 10  LOC LAT                        1121 non-null   float64 
 11  LOC LONG                       1121 non-null   float64 
 12  LOC Center LAT

#### - TRAIN & TEST (BID/OFFER MODEL)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
X_train = DataTrain_df.drop(axis=1, columns='acceptedprice', level=None, inplace=False, errors='raise')
y_train = DataTrain_df['acceptedprice']
print(X_train.shape)
print(y_train.shape)

(1191636, 91)
(1191636,)


In [ ]:
X_train

,recordtype,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,BID,2021-01-01,1,-18.366,68.54,407.30,E,CCGT,Eastern England,52.583834,...,1,1,0,0,1,0,0,0,0,0
1,BID,2021-01-01,1,-0.666,68.54,407.30,E,CCGT,South Eastern England,50.829511,...,1,1,0,0,1,0,0,0,0,0
2,OFFER,2021-01-01,1,248.334,68.54,407.30,T,CCGT,North Western England,53.436959,...,1,1,0,0,1,0,0,0,0,0
3,BID,2021-01-01,1,-11.084,68.54,407.30,T,CCGT,East Midlands,53.307421,...,1,1,0,0,1,0,0,0,0,0
4,OFFER,2021-01-01,1,322.584,68.54,407.30,T,CCGT,East Midlands,53.307421,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191631,OFFER,2023-06-29,48,120.000,74.72,770.85,T,CCGT,South Western England,51.539486,...,1,1,0,0,1,908,47,2,5,28
1191632,BID,2023-06-29,48,-84.404,74.72,770.85,T,CCGT,Yorkshire,53.602454,...,1,1,0,0,1,908,47,2,5,28
1191633,BID,2023-06-29,48,-7.009,74.72,770.85,T,WIND,Northern Scotland,58.510313,...,1,1,0,0,1,908,47,2,5,28
1191634,BID,2023-06-29,48,-2.486,74.72,770.85,T,WIND,Southern Scotland,55.347448,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
y_train

0           37.00
1           35.00
2           74.90
3           40.00
4           71.90
            ...  
1191631    160.00
1191632     62.00
1191633    -71.77
1191634    -75.00
1191635    -75.00
Name: acceptedprice, Length: 1191636, dtype: float64

In [ ]:
X_test = DataTest_df.drop(axis=1, columns='acceptedprice', level=None, inplace=False, errors='raise')
#X_test.reset_index(level=None, drop=True, inplace=True)
y_test = DataTest_df['acceptedprice']
#y_test.reset_index(level=None, drop=True, inplace=True)
print(X_test.shape)
print(y_test.shape)

(1121, 91)
(1121,)


In [ ]:
X_test.head(5)

,recordtype,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191636,OFFER,2023-06-30,1,8.167,69.81,1072.4,2,BATTERY,East Midlands,52.800048,...,1,0,0,1,1,909,0,2,5,29
1191637,BID,2023-06-30,1,-2.950,69.81,1072.4,E,WIND,Northern Scotland,58.568706,...,2,1,1,1,1,909,0,2,5,29
1191638,BID,2023-06-30,1,-1.000,69.81,1072.4,E,WIND,Southern Scotland,55.569968,...,0,0,1,1,1,909,0,2,5,29
1191639,BID,2023-06-30,1,-4.492,69.81,1072.4,E,WIND,Northern Scotland,55.555764,...,2,1,1,1,1,909,0,2,5,29
1191640,BID,2023-06-30,1,-2.500,69.81,1072.4,E,WIND,Northern Scotland,57.363367,...,0,0,1,1,1,909,0,2,5,29


#### - TRAIN & TEST (BID/OFFER MODEL - BID)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
Xbid_train = DataTrain_df[DataTrain_df['recordtype']=='BID']
Xbid_train = Xbid_train.drop(axis=1, columns=['recordtype', 'recordtype_label','acceptedprice'], level=None, inplace=False, errors='raise')
ybid_train = DataTrain_df[DataTrain_df['recordtype']=='BID']['acceptedprice']
print(Xbid_train.shape)
print(ybid_train.shape)

(745115, 89)
(745115,)


In [ ]:
Xbid_train

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,2021-01-01,1,-18.366,68.54,407.30,E,CCGT,Eastern England,52.583834,1.733725,...,1,1,0,0,1,0,0,0,0,0
1,2021-01-01,1,-0.666,68.54,407.30,E,CCGT,South Eastern England,50.829511,-0.229161,...,1,1,0,0,1,0,0,0,0,0
3,2021-01-01,1,-11.084,68.54,407.30,T,CCGT,East Midlands,53.307421,-0.786058,...,1,1,0,0,1,0,0,0,0,0
10,2021-01-01,1,-174.768,68.54,407.30,T,CCGT,Southern England,50.898831,-1.437187,...,1,1,0,0,1,0,0,0,0,0
12,2021-01-01,1,-33.634,68.54,407.30,T,CCGT,Southern Wales,51.683003,-4.994865,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191630,2023-06-29,48,-11.086,74.72,770.85,T,WIND,Southern Scotland,55.368624,-4.042596,...,1,1,0,0,1,908,47,2,5,28
1191632,2023-06-29,48,-84.404,74.72,770.85,T,CCGT,Yorkshire,53.602454,-0.144829,...,1,1,0,0,1,908,47,2,5,28
1191633,2023-06-29,48,-7.009,74.72,770.85,T,WIND,Northern Scotland,58.510313,-4.021024,...,1,1,0,0,1,908,47,2,5,28
1191634,2023-06-29,48,-2.486,74.72,770.85,T,WIND,Southern Scotland,55.347448,-3.880540,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
ybid_train

0          37.00
1          35.00
3          40.00
10         37.10
12         37.00
           ...  
1191630   -75.00
1191632    62.00
1191633   -71.77
1191634   -75.00
1191635   -75.00
Name: acceptedprice, Length: 745115, dtype: float64

In [ ]:
Xbid_test = DataTest_df[DataTest_df['recordtype']=='BID']
Xbid_test = Xbid_test.drop(axis=1, columns=['recordtype','acceptedprice'], level=None, inplace=False, errors='raise')
ybid_test = DataTest_df[DataTest_df['recordtype']=='BID']['acceptedprice']
print(Xbid_test.shape)
print(ybid_test.shape)

(875, 90)
(875,)


In [ ]:
Xbid_test.head(5)

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191637,2023-06-30,1,-2.950,69.81,1072.4,E,WIND,Northern Scotland,58.568706,-3.676014,...,2,1,1,1,1,909,0,2,5,29
1191638,2023-06-30,1,-1.000,69.81,1072.4,E,WIND,Southern Scotland,55.569968,-5.622702,...,0,0,1,1,1,909,0,2,5,29
1191639,2023-06-30,1,-4.492,69.81,1072.4,E,WIND,Northern Scotland,55.555764,-5.576596,...,2,1,1,1,1,909,0,2,5,29
1191640,2023-06-30,1,-2.500,69.81,1072.4,E,WIND,Northern Scotland,57.363367,-2.963840,...,0,0,1,1,1,909,0,2,5,29
1191642,2023-06-30,1,-45.000,69.81,1072.4,E,CCGT,South Eastern England,50.829511,-0.229161,...,1,1,0,0,1,909,0,2,5,29


In [ ]:
ybid_test

1191637   -71.50
1191638   -71.50
1191639   -21.53
1191640   -68.00
1191642    63.11
           ...  
1192752   -79.33
1192753   -75.00
1192754   -75.00
1192755   -80.00
1192756    25.00
Name: acceptedprice, Length: 875, dtype: float64

#### - TRAIN & TEST (BID/OFFER MODEL - OFFER)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
Xoffer_train = DataTrain_df[DataTrain_df['recordtype']=='OFFER']
Xoffer_train = Xoffer_train.drop(axis=1, columns=['recordtype', 'recordtype_label','acceptedprice'], level=None, inplace=False, errors='raise')
yoffer_train = DataTrain_df[DataTrain_df['recordtype']=='OFFER']['acceptedprice']
print(Xoffer_train.shape)
print(yoffer_train.shape)

(446521, 89)
(446521,)


In [ ]:
Xoffer_train

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
2,2021-01-01,1,248.334,68.54,407.30,T,CCGT,North Western England,53.436959,-2.408212,...,1,1,0,0,1,0,0,0,0,0
4,2021-01-01,1,322.584,68.54,407.30,T,CCGT,East Midlands,53.307421,-0.786058,...,1,1,0,0,1,0,0,0,0,0
5,2021-01-01,1,270.000,68.54,407.30,T,CCGT,South Eastern England,51.425650,0.600555,...,1,1,0,0,1,0,0,0,0,0
6,2021-01-01,1,230.188,68.54,407.30,T,CCGT,South Eastern England,51.443546,0.707775,...,1,1,0,0,1,0,0,0,0,0
7,2021-01-01,1,230.000,68.54,407.30,T,CCGT,South Eastern England,51.443546,0.707775,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191573,2023-06-29,47,0.171,78.80,831.45,T,CCGT,Yorkshire,53.639914,-0.234288,...,1,1,0,0,1,908,46,2,5,28
1191586,2023-06-29,47,115.000,78.80,831.45,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,908,46,2,5,28
1191588,2023-06-29,47,120.000,78.80,831.45,T,CCGT,South Western England,51.539486,-2.670153,...,1,1,0,0,1,908,46,2,5,28
1191629,2023-06-29,48,115.000,74.72,770.85,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
yoffer_train

2           74.9
4           71.9
5           72.0
6           71.9
7           70.0
           ...  
1191573    169.0
1191586    165.0
1191588    160.0
1191629    165.0
1191631    160.0
Name: acceptedprice, Length: 446521, dtype: float64

In [ ]:
Xoffer_test = DataTest_df[DataTest_df['recordtype']=='OFFER']
Xoffer_test = Xoffer_test.drop(axis=1, columns=['recordtype','acceptedprice'], level=None, inplace=False, errors='raise')
yoffer_test = DataTest_df[DataTest_df['recordtype']=='OFFER']['acceptedprice']
print(Xoffer_test.shape)
print(yoffer_test.shape)

(246, 90)
(246,)


In [ ]:
Xoffer_test.head(5)

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191636,2023-06-30,1,8.167,69.81,1072.4,2,BATTERY,East Midlands,52.800048,-0.927751,...,1,0,0,1,1,909,0,2,5,29
1191641,2023-06-30,1,0.900,69.81,1072.4,E,BATTERY,Yorkshire,53.799900,-0.410000,...,2,1,1,1,1,909,0,2,5,29
1191653,2023-06-30,1,12.000,69.81,1072.4,T,PS,Merseyside and Northern Wales,52.980956,-3.968825,...,1,1,0,0,1,909,0,2,5,29
1191654,2023-06-30,1,12.000,69.81,1072.4,T,PS,Merseyside and Northern Wales,52.980956,-3.968825,...,1,1,0,0,1,909,0,2,5,29
1191667,2023-06-30,1,115.000,69.81,1072.4,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,909,0,2,5,29


In [ ]:
yoffer_test

1191636    119.36
1191641    119.00
1191653    159.75
1191654    159.90
1191667    165.00
            ...  
1192742    140.00
1192744    165.00
1192745    160.00
1192747    175.00
1192750    120.00
Name: acceptedprice, Length: 246, dtype: float64

## 3. XGBOOST (HYPERPARAMETERS)


**Control Overfitting**

When you observe high training accuracy, but low test accuracy, it is likely that you encountered overfitting problem. There are in general two ways that you can control overfitting in XGBoost:

- The first way is to directly control model complexity.

 - This includes max_depth, min_child_weight and gamma.

- The second way is to add randomness to make training robust to noise.

 - This includes subsample and colsample_bytree.

 - You can also reduce stepsize eta. Remember to increase num_round when you do so.

**Faster training performance**

There is a parameter called tree_method, set it to hist or gpu_hist for faster computation.


    params = {
      'colsample_bynode': 0.8,
      'learning_rate': 1,
      'max_depth': 5,
      'num_parallel_tree': 100,
      'objective': 'binary:logistic',
      'subsample': 0.8,
      'tree_method': 'gpu_hist'
    }


### Hiperparámetros

In [ ]:
        "n_estimators": trial.suggest_int(name="n_estimators", low=10, high=100, log=True),
        "min_samples_split": 1000,
        "max_depth":25,
        "n_jobs": -1,
        "random_state": 0

In [ ]:
        "min_samples_split": trial.suggest_int(name="min_samples_split", low=2, high=15),
        "max_features":  trial.suggest_int(name="max_features", low=8, high=60),
        "max_depth": trial.suggest_int(name="max_depth", low=1, high=800, log=True),
        "n_estimators": trial.suggest_int(name="n_estimators", low=1, high=2000, log=True),
        #"bootstrap": True,
        #"max_samples": trial.suggest_float(name="max_samples", low=0.5, high=1, log=True),
        "n_jobs": -1,
        "random_state": 314

In [ ]:
# ANÁLISIS HIPERPARÁMETROS CLASE MACHINE LEARNING
# INCREASE PREDICTIVE POWER
"n_estimators"
"max_features"
"mini_sample_leaf"
# INCREASE MODEL SPEED
"n_jobs"
"random_state"
"oob_score":True/False

In [ ]:
.oob_score_

In [ ]:
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1),
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=2, scoring="r2")

In [ ]:
best_forest=grid.best_estimator_
best_forest

'''
RandomForestClassifier
RandomForestClassifier(max_depth=5, min_samples_leaf=5, n_estimators=50,
                       n_jobs=-1, random_state=42)
'''

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_train.columns,
    "Imp": best_forest.feature_importances_
})

In [ ]:
imp_df.sort_values(by="Imp", ascending=False)

## 4. XGBOOST (BID/OFFER MODEL)

    VIF
      ['DAIG_spnGeneration',
      'DANF_Demand',
      'DATF_Demand',
      'LOC Center LAT',
      'LOC LAT',
      'SystemTotalLoad_Consumption',
      'LOC Center LONG',
      'GenerationForecast_Production',
      'NatGas_price']

### Modelo BID (Predicción diaria)

In [ ]:
#test_dates = ['01','04','05','08','12','15','18','19','22','25','26','28','30']
test_dates = ['10','15','29']
recordtype = 'BID'

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x

  train = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_indxs = list(train.index.values)

  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)].copy()
  test_indxs = list(test.index.values)


  train_test[x] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['10', '15', '29'])

In [ ]:
train_test['10'].keys()

dict_keys(['train_indxs', 'test_indxs'])

In [ ]:
for x in list(train_test.keys()):
  print(len(train_test[x]['train_indxs']))
  print(len(train_test[x]['test_indxs']))

733852
487
736348
420
744275
840


In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume',
 'marketIndexPrice',
 'marketIndexVolume',
 'LOC LAT',
 'LOC LONG']

In [ ]:
X_train = df_model_label.loc[train_test['10']['train_indxs'], select_vars]
y_train = df_model_label.loc[train_test['10']['train_indxs'], 'acceptedprice']
print(X_train.shape)

X_test = df_model_label.loc[train_test['10']['test_indxs'], select_vars]
y_test = df_model_label.loc[train_test['10']['test_indxs'],'acceptedprice']
print(X_test.shape)

(733852, 73)
(487, 73)


In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
train_scores = []
test_scores = []


for x in test_dates:

  start = time.time()

  test_d = '2023-06-' + x
  test_days.append(test_d)
  print('_'*35)
  print(f'Model XGBoost (Test Day {x})')

  X_train = df_model_label.loc[train_test[x]['train_indxs'], select_vars]
  y_train = df_model_label.loc[train_test[x]['train_indxs'], 'acceptedprice']

  X_test = df_model_label.loc[train_test[x]['test_indxs'], select_vars]
  y_test = df_model_label.loc[train_test[x]['test_indxs'],'acceptedprice']


  RF_bid.fit(X_train, y_train)


  size_train = y_train.shape[0]
  sumresid_train = sum(y_train - RF_bid.predict(X_train))
  stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
  r2_train = r2_score(y_train, RF_bid.predict(X_train))
  rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
  train_scores.append(rmse_train)
  print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
  print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

  size_test = y_test.shape[0]
  sumresid_test = sum(y_test - RF_bid.predict(X_test))
  stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
  r2_test = r2_score(y_test, RF_bid.predict(X_test))
  rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
  test_scores.append(rmse_test)
  print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
  print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


  model_imp = 'imp_' + x
  model_importances[model_imp] = RF_bid.feature_importances_

  end = time.time()
  print(timedelta(minutes=end-start))
  #print('\n')

model_scores = {'test_days':test_days,'train_scores':train_scores,'test_scores':test_scores}


___________________________________
Model XGBoost (Test Day 10)
Train scores: Score=0.9653625048632484 / R2=0.9653625048632484 / RMSE=19.25601400773443
Test scores: Score=0.835090627386625 / R2=0.835090627386625 / RMSE=20.80419839595804
1:33:35.667086
___________________________________
Model XGBoost (Test Day 15)
Train scores: Score=0.9624852328794156 / R2=0.9624852328794156 / RMSE=20.135265991350902
Test scores: Score=-35.3778572348729 / R2=-35.3778572348729 / RMSE=132.73083947344185
1:28:31.110921
___________________________________
Model XGBoost (Test Day 29)


KeyboardInterrupt: ignored

In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum,imp_10,imp_15,imp_29
53,sp3_6_pavgprice,0.271756,0.297801,0.255150,0.262316
60,BMU Fuel Type_label,0.157583,0.153737,0.159112,0.159898
59,BMU Type_label,0.043761,0.012959,0.075573,0.042751
4,LOC LONG,0.039225,0.042140,0.040583,0.034953
9,Z1,0.037282,0.038867,0.037395,0.035584
...,...,...,...,...,...
32,Weekday_COS,0.000801,0.000940,0.000695,0.000767
25,Z17,0.000624,0.000112,0.000550,0.001210
18,Z10,0.000523,0.000694,0.000805,0.000070
23,Z15,0.000387,0.000000,0.001160,0.000000


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_10,imp_15,imp_29
53,sp3_6_pavgprice,0.271756,0.297801,0.255150,0.262316
60,BMU Fuel Type_label,0.157583,0.153737,0.159112,0.159898
59,BMU Type_label,0.043761,0.012959,0.075573,0.042751
4,LOC LONG,0.039225,0.042140,0.040583,0.034953
9,Z1,0.037282,0.038867,0.037395,0.035584
26,Transmission Loss Factor,0.032935,0.038721,0.014983,0.045100
2,marketIndexVolume,0.022887,0.015203,0.031299,0.022158
56,sp6_12_pavgprice,0.021532,0.019125,0.017909,0.027563
6,LOC Center LONG,0.021353,0.019814,0.025512,0.018732
66,Exempt Export Flag_label,0.021235,0.017913,0.018405,0.027386


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_10,imp_15,imp_29
13,Z5,0.001684,0.001936,0.001469,0.001646
5,LOC Center LAT,0.001571,0.002369,0.001233,0.001113
22,Z14,0.001562,0.001751,0.001449,0.001486
46,GenerationForecast_Production,0.001417,0.000915,0.001134,0.002202
17,Z9,0.001415,0.001116,0.001113,0.002016
58,sp6_12_totalvolume,0.001353,0.001617,0.001176,0.001265
67,Base TU Flag_label,0.001302,0.001145,0.002130,0.000630
15,Z7,0.001282,0.000735,0.001809,0.001303
29,Weekend,0.001281,0.000741,0.000998,0.002104
42,DATF_Demand,0.001245,0.001163,0.000744,0.001828


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo BID (Predicción por Mañana/Tarde)

In [ ]:
recordtype = 'BID'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
test_dates = ['01','06','10','15','20','24','29']
test_periods = ['Mañana SP1-24','Tarde SP25-48']

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}


  test_sp = 'Mañana SP1-24'

  train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_indxs = list(train_prev.index.values)
  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<=24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}



  test_sp = 'Tarde SP25-48'

  train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<=24)].copy()
  train = pd.concat([train_prev, train_day])
  train_indxs = list(train.index.values)

  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']>24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-06', '2023-06-10', '2023-06-15', '2023-06-20', '2023-06-24', '2023-06-29'])

In [ ]:
x = '2023-06-01'
print(train_test[x].keys())

dict_keys(['Mañana SP1-24', 'Tarde SP25-48'])


In [ ]:
x = '2023-06-01'
y = 'Mañana SP1-24'
#y = 'Tarde SP25-48'

In [ ]:
train = df_model_label.loc[train_test[x][y]['train_indxs'],:]
print(train.shape)
train

(729536, 90)


,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,37.0,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3,29,1,1,0,0,1,0,0,0
1,BID,2021-01-01,1,35.0,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,0,0,0
3,BID,2021-01-01,1,40.0,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
10,BID,2021-01-01,1,37.1,-174.768,T,CCGT,Southern England,50.898831,-1.437187,...,7,29,1,1,0,0,1,0,0,0
12,BID,2021-01-01,1,37.0,-33.634,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161221,BID,2023-05-31,48,26.0,-38.795,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,2,4,30
1161222,BID,2023-05-31,48,30.5,-4.000,T,CCGT,South Western England,51.539486,-2.670153,...,4,36,1,1,0,0,1,2,4,30
1161224,BID,2023-05-31,48,20.5,-49.556,T,CCGT,Yorkshire,53.360271,-0.813505,...,1,38,1,1,0,0,1,2,4,30
1161225,BID,2023-05-31,48,20.5,-21.044,T,CCGT,Yorkshire,53.360271,-0.813505,...,15,38,1,1,0,0,1,2,4,30


In [ ]:
test = df_model_label.loc[train_test[x][y]['test_indxs'],:]
print(test.shape)
test

(264, 90)


,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
1161227,BID,2023-06-01,1,31.33,-57.764,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,2,5,0
1161229,BID,2023-06-01,1,31.30,-99.500,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,2,5,0
1161231,BID,2023-06-01,1,31.30,-36.000,T,CCGT,South Eastern England,51.443546,0.707775,...,6,25,1,1,0,0,1,2,5,0
1161232,BID,2023-06-01,1,31.30,-29.000,T,CCGT,South Eastern England,51.443546,0.707775,...,6,25,1,1,0,0,1,2,5,0
1161233,BID,2023-06-01,1,24.00,-20.248,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,2,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161611,BID,2023-06-01,24,75.00,-12.667,T,CCGT,Yorkshire,53.735187,-0.243281,...,15,34,1,1,0,0,1,2,5,0
1161613,BID,2023-06-01,24,33.05,-23.608,T,CCGT,Yorkshire,53.360271,-0.813505,...,1,38,1,1,0,0,1,2,5,0
1161614,BID,2023-06-01,24,33.05,-10.336,T,CCGT,Yorkshire,53.360271,-0.813505,...,15,38,1,1,0,0,1,2,5,0
1161615,BID,2023-06-01,24,51.00,-0.333,V,BATTERY,North Western England,53.780364,-2.642276,...,16,29,0,0,0,0,1,2,5,0


In [ ]:
train_test[x][y]['train_indxs'][-1]

1161226

In [ ]:
train_test[x][y]['test_indxs'][-1]

1161616

In [ ]:
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

for x in test_dates:
  for y in test_periods:

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')
    print(len(train_test[x][y]['train_indxs']))
    print(len(train_test[x][y]['test_indxs']))

_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Mañana SP1-24)
729536
264
_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Tarde SP25-48)
729800
340
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Mañana SP1-24)
732078
182
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Tarde SP25-48)
732260
133
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Mañana SP1-24)
733852
253
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Tarde SP25-48)
734105
234
_______________________________________________________
Model XGBoost (Test: Day 2023-06-15 / Mañana SP1-24)
736348
247
_______________________________________________________
Model XGBoost (Test: Day 2023-06-15 / Tarde SP25-48)
736595
173
________________________________________

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume', 'LOC LAT', 'LOC LONG', 'LOC Center LONG', 'Z1']

In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-06', '2023-06-10', '2023-06-15', '2023-06-20', '2023-06-24', '2023-06-29'])

In [ ]:
train_test['2023-06-01'].keys()

dict_keys(['Mañana SP1-24', 'Tarde SP25-48'])

In [ ]:
train_test['2023-06-01']['Mañana SP1-24'].keys()

dict_keys(['train_indxs', 'test_indxs'])

In [ ]:
X_train = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], select_vars]
y_train = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], 'acceptedprice']
print(X_train.shape)

X_test = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'], select_vars]
y_test = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'],'acceptedprice']
print(X_test.shape)

(729536, 28)
(264, 28)


In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
test_sps = []
train_scores = []
test_scores = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    sumresid_train = sum(y_train - RF_bid.predict(X_train))
    stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
    r2_train = r2_score(y_train, RF_bid.predict(X_train))
    rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
    train_scores.append(rmse_train)
    print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
    print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

    size_test = y_test.shape[0]
    sumresid_test = sum(y_test - RF_bid.predict(X_test))
    stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
    r2_test = r2_score(y_test, RF_bid.predict(X_test))
    rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
    test_scores.append(rmse_test)
    print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
    print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


    if y == 'Mañana SP1-24':
      model_imp = 'imp_' + x + 'M'
    else:
      model_imp = 'imp_' + x + 'T'
    model_importances[model_imp] = RF_bid.feature_importances_

    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_days':test_days,'test_sps':test_sps,'train_scores':train_scores,'test_scores':test_scores}


_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Mañana SP1-24)
0.8869290952241193
-1.2533265130998092
0:27:21.786919
_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Tarde SP25-48)
0.8982082635778457
-0.9201004950384222
0:26:56.863589
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Mañana SP1-24)
0.8994777606947875
-1.333947801934912
0:27:20.676212
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Tarde SP25-48)
0.8834751388163431
-0.07634411110709971
0:26:47.568913
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Mañana SP1-24)
0.8840903058677078
0.7222504522673059
0:26:43.569989
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Tarde SP25-48)
0.9076883169864725
0.8099592267000579
0:26:55.117679
________________________

In [ ]:
#pd.DataFrame(model_scores)

,test_days,test_sps,train_scores,test_scores
0,2023-06-01,Mañana SP1-24,0.953246,-0.518197
1,2023-06-01,Tarde SP25-48,0.953329,-0.818559
2,2023-06-06,Mañana SP1-24,0.954304,-7.902843
3,2023-06-06,Tarde SP25-48,0.955386,-0.334653
4,2023-06-10,Mañana SP1-24,0.955892,0.778999
5,2023-06-10,Tarde SP25-48,0.954294,0.817314
6,2023-06-15,Mañana SP1-24,0.953824,-0.985459
7,2023-06-15,Tarde SP25-48,0.954184,-21.364026
8,2023-06-20,Mañana SP1-24,0.949789,0.137927
9,2023-06-20,Tarde SP25-48,0.952392,-2.351894


In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

In [ ]:
model_imp_df.head(20)

In [ ]:
model_imp_df.tail(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo BID (Predicción por SP)

In [ ]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-03', '2023-06-05', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-13', '2023-06-15', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-23', '2023-06-25', '2023-06-27', '2023-06-29'])

In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

729564
10
729625
10
729707
9
729760
10
729800
10
729861
12
729933
17
729977
25
730132
8
730718
11
730803
20
730890
13
730973
14
731008
8
731041
6
731111
23
731158
7
731216
3
731526
8
731574
14
731654
6
731693
3
731707
9
731769
19
731852
15
731889
19
732072
6
732425
9
732476
7
732558
9
732589
0
732589
6
732642
10
732707
4
732718
4
732798
14
733450
11
733494
6
733564
9
733631
10
733652
6
733690
4
733740
10
733765
5
733842
10
734403
4
734415
0
734460
10
734520
15
734549
5
734612
6
734634
13
734654
13
734709
4
735397
17
735463
12
735544
38
735669
17
735714
15
735776
5
735830
11
735863
10
735972
8
736359
5
736379
1
736465
12
736564
14
736595
3
736628
15
736684
5
736689
1
736759
9
737216
2
737244
4
737297
7
737359
10
737392
10
737468
9
737534
10
737564
7
737635
7
738155
7
738218
12
738341
17
738411
7
738432
6
738496
20
738593
21
738680
38
738927
17
739555
7
739601
15
739708
12
739787
5
739807
12
739885
11
739950
12
740009
7
740088
12
740617
14
740693
13
740788
11
740826
15
740865
10
740954
2

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume',
 'marketIndexPrice',
 'marketIndexVolume',
 'LOC LAT',
 'LOC LONG']

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=729564 / RMSE=19.834252955941228 / MeanReal: 30.365312943004458 / MeanPred: 30.36615753173828
              Resid=-613.3440596297435 / AbsResidMean=10.854628226958118 / ResidMean=-0.0008406994583538925 / StdResidMean=2.718644688899697e-05
Test scores: N=10 / RMSE=7.544216819262409 / MeanReal: 30.019 / MeanPred: 25.44683837890625
             Resid=45.7216047668457 / AbsResidMean=6.86064616394043 / ResidMean=4.57216047668457 / StdResidMean=0.6000879601482213
1:24:33.767052
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=729625 / RMSE=19.72717202683329 / MeanReal: 30.365394719136688 / MeanPred: 30.366559982299805
              Resid=-847.7919506816387 / AbsResidMean=10.85969436235469 / ResidMean=-0.0011619557316140546 / StdResidMean=2.7037412359243376e-05
Test scores: N=10 / RMSE=12.486856194284488 / MeanReal: 32.025999999999

In [ ]:
pd.DataFrame(model_scores)

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,729564,19.834253,30.365313,30.365313,-613.344060,10.854628,-0.000841,0.000027,10,7.544217,30.019000,25.446838,45.721605,6.860646,4.572160,0.600088
1,2023-06-01,10,729625,19.727172,30.365395,30.365395,-847.791951,10.859694,-0.001162,0.000027,10,12.486856,32.026000,24.257334,77.686651,8.102976,7.768665,0.977596
2,2023-06-01,16,729707,20.060421,30.365595,30.365595,-154.835440,10.824469,-0.000212,0.000027,9,5.485145,29.983333,28.911425,9.647190,4.079988,1.071910,0.597710
3,2023-06-01,22,729760,19.746984,30.365725,30.365725,1.601949,10.755592,0.000002,0.000027,10,17.269947,34.290000,34.540947,-2.509437,12.659650,-0.250944,1.726812
4,2023-06-01,25,729800,19.713087,30.366161,30.366161,-240.244610,10.662194,-0.000329,0.000027,10,17.636628,33.840000,29.082016,47.579855,10.768467,4.757986,1.698270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,744665,20.127468,30.626629,30.626629,-189.502099,10.838168,-0.000254,0.000027,16,7.711669,46.303750,40.799549,88.067221,6.020615,5.504201,0.337579
131,2023-06-29,31,744786,20.454417,30.629706,30.629706,164.114251,10.841292,0.000220,0.000027,20,13.181766,49.814500,42.195854,152.373035,9.637564,7.618652,0.537855
132,2023-06-29,37,744900,20.626620,30.632497,30.632497,81.511509,10.818635,0.000109,0.000028,13,4.978444,43.133077,40.906860,28.940846,3.253132,2.226219,0.342535
133,2023-06-29,40,744931,20.211288,30.632818,30.632818,-350.172513,10.715901,-0.000470,0.000027,11,3.707254,40.043636,41.352184,-14.394030,3.059188,-1.308548,0.315331


In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID MODEL - SP (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID MODEL - SP (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
53,sp3_6_pavgprice,0.257274,0.263618,0.253825,0.285525,0.283383,0.290455,0.266927,0.259419,0.259248,0.274093,0.175948,0.253892,0.231438,0.275092,0.273781,0.212472
60,BMU Fuel Type_label,0.140617,0.133548,0.130314,0.152260,0.151769,0.161132,0.144158,0.150603,0.144445,0.153664,0.100488,0.147165,0.138228,0.135639,0.131873,0.133970
9,Z1,0.039484,0.044905,0.042126,0.042988,0.033967,0.050611,0.045969,0.039658,0.034544,0.039179,0.027569,0.044857,0.031874,0.034496,0.040824,0.038694
59,BMU Type_label,0.039071,0.033902,0.012191,0.027966,0.039943,0.010829,0.028616,0.105040,0.075918,0.065378,0.010391,0.048448,0.025427,0.041386,0.040875,0.019756
26,Transmission Loss Factor,0.038942,0.049034,0.037407,0.054825,0.056965,0.036271,0.016101,0.012702,0.029236,0.037198,0.006731,0.041211,0.052520,0.055746,0.054348,0.043833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,settlementPeriod_ordin,0.001008,0.000889,0.001142,0.000485,0.000678,0.000825,0.000784,0.004325,0.000707,0.000835,0.000439,0.000748,0.001066,0.000657,0.000866,0.000667
42,DATF_Demand,0.000887,0.001209,0.000889,0.000996,0.000942,0.001012,0.000817,0.000732,0.000455,0.001164,0.000574,0.000962,0.000789,0.001003,0.000867,0.000895
25,Z17,0.000693,0.001053,0.000000,0.000000,0.000000,0.000905,0.000684,0.000261,0.000276,0.000433,0.000234,0.000000,0.000000,0.000000,0.000537,0.006010
23,Z15,0.000228,0.000566,0.000000,0.000000,0.000000,0.000222,0.000230,0.000000,0.000000,0.000000,0.000000,0.000700,0.000000,0.001696,0.000000,0.000000


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
53,sp3_6_pavgprice,0.257274,0.263618,0.253825,0.285525,0.283383,0.290455,0.266927,0.259419,0.259248,0.274093,0.175948,0.253892,0.231438,0.275092,0.273781,0.212472
60,BMU Fuel Type_label,0.140617,0.133548,0.130314,0.152260,0.151769,0.161132,0.144158,0.150603,0.144445,0.153664,0.100488,0.147165,0.138228,0.135639,0.131873,0.133970
9,Z1,0.039484,0.044905,0.042126,0.042988,0.033967,0.050611,0.045969,0.039658,0.034544,0.039179,0.027569,0.044857,0.031874,0.034496,0.040824,0.038694
59,BMU Type_label,0.039071,0.033902,0.012191,0.027966,0.039943,0.010829,0.028616,0.105040,0.075918,0.065378,0.010391,0.048448,0.025427,0.041386,0.040875,0.019756
26,Transmission Loss Factor,0.038942,0.049034,0.037407,0.054825,0.056965,0.036271,0.016101,0.012702,0.029236,0.037198,0.006731,0.041211,0.052520,0.055746,0.054348,0.043833
4,LOC LONG,0.036210,0.015388,0.037086,0.041666,0.028132,0.040791,0.037171,0.032901,0.055671,0.040394,0.034509,0.043526,0.034511,0.029376,0.031933,0.040102
29,Weekend,0.033708,0.001000,0.002647,0.000544,0.000758,0.000249,0.000483,0.024563,0.001128,0.000261,0.278619,0.001190,0.050826,0.002316,0.000917,0.140119
6,LOC Center LONG,0.025020,0.022393,0.026102,0.020748,0.025729,0.022463,0.023158,0.024649,0.013842,0.046807,0.037239,0.032166,0.019646,0.025095,0.022553,0.012714
56,sp6_12_pavgprice,0.021905,0.023816,0.025991,0.024310,0.022724,0.019578,0.027560,0.012453,0.014429,0.024316,0.016085,0.024247,0.026376,0.021634,0.020750,0.024299
66,Exempt Export Flag_label,0.018350,0.017975,0.018924,0.017296,0.020100,0.015261,0.028189,0.021682,0.013190,0.017616,0.010704,0.023166,0.016526,0.020959,0.013189,0.020477


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
30,Holiday,0.001876,0.001236,0.001379,0.001696,0.002810,0.004096,0.004457,0.004966,0.000681,0.000835,0.001145,0.000927,0.001001,0.001348,0.001190,0.000370
13,Z5,0.001858,0.002129,0.002004,0.002553,0.002102,0.002064,0.001491,0.002651,0.001309,0.001494,0.001262,0.001666,0.001901,0.001720,0.001737,0.001785
16,Z8,0.001694,0.000959,0.000449,0.000860,0.001950,0.002197,0.000927,0.001445,0.001586,0.000472,0.009470,0.000882,0.001111,0.001340,0.000932,0.000826
5,LOC Center LAT,0.001675,0.004562,0.001267,0.001004,0.002208,0.002716,0.001170,0.001384,0.001408,0.001327,0.000551,0.001262,0.002597,0.001101,0.001429,0.001139
21,Z13,0.001662,0.000695,0.001545,0.002486,0.001724,0.001362,0.001659,0.001580,0.002092,0.001421,0.000611,0.002947,0.002069,0.002261,0.001853,0.000629
31,Weekday_SIN,0.001521,0.000583,0.006099,0.000658,0.003237,0.000704,0.000939,0.000774,0.000883,0.001087,0.001247,0.000833,0.000599,0.000961,0.000442,0.003768
8,BZONE DEMAND,0.001502,0.001967,0.000911,0.000821,0.001002,0.002555,0.001150,0.001767,0.003232,0.000864,0.001477,0.001247,0.001679,0.001155,0.000943,0.001761
36,Day_COS,0.001457,0.000927,0.001297,0.001214,0.001279,0.001051,0.001253,0.002382,0.001518,0.001079,0.001058,0.002015,0.001728,0.000960,0.001980,0.002109
18,Z10,0.001288,0.000558,0.001651,0.000385,0.001179,0.000259,0.004239,0.000290,0.000159,0.000729,0.000264,0.000061,0.006654,0.001915,0.000579,0.000401
61,BMU GSP Group Name_label,0.001281,0.001103,0.000865,0.001301,0.001305,0.000996,0.001113,0.001089,0.001932,0.001504,0.001878,0.000907,0.001225,0.001209,0.001705,0.001086


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

## 5. XGBOOST (SP PAVG. PRICE MODEL)

    VIF
      ['DAIG_spnGeneration',
      'DANF_Demand',
      'DATF_Demand',
      'SystemTotalLoad_Consumption',
      'GenerationForecast_Production',
      'NatGas_price']

### Modelo BID (Predicción diaria)

In [ ]:
df_modelAVG_label.head()

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,marketIndexPrice,marketIndexVolume,LOC Center LAT,LOC Center LONG,Weekend,...,sp36_48_pavgprice,sp36_48_meanvolume,sp36_48_totalvolume,recordtype_label,BMU GSP Group Name_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,BID,2021-01-01,1,East Midlands,40.0,68.54,407.30,52.800048,-0.927751,0.0,...,33.812282,-0.040333,-0.040333,0,0,0,0,0,0,0
1,OFFER,2021-01-01,1,East Midlands,71.9,68.54,407.30,52.800048,-0.927751,0.0,...,68.501114,104.789792,158.941333,1,0,0,0,0,0,0
2,OFFER,2021-01-01,2,East Midlands,71.9,69.82,601.40,52.800048,-0.927751,0.0,...,68.531072,109.552083,168.465917,1,0,0,1,0,0,0
3,OFFER,2021-01-01,3,East Midlands,71.9,67.42,608.00,52.800048,-0.927751,0.0,...,68.658646,105.195042,171.990667,1,0,0,2,0,0,0
4,OFFER,2021-01-01,4,East Midlands,71.9,65.45,579.35,52.800048,-0.927751,0.0,...,68.944868,109.481375,170.980000,1,0,0,3,0,0,0


In [ ]:
#test_dates = ['01','04','05','08','12','15','18','19','22','25','26','28','30']
test_dates = ['10','15','29']

recordtype = 'BID'

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x

  train = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_indxs = list(train.index.values)

  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)].copy()
  test_indxs = list(test.index.values)


  train_test[x] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['10', '15', '29'])

In [ ]:
for x in list(train_test.keys()):
  print(len(train_test[x]['train_indxs']))
  print(len(train_test[x]['test_indxs']))

233338
188
234607
247
238394
360


In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['marketIndexPrice',
 'marketIndexVolume',
 'LOC Center LAT',
 'LOC Center LONG',
 'Weekend']

In [ ]:
X_train = df_modelAVG_label.loc[train_test['10']['train_indxs'], select_vars]
y_train = df_modelAVG_label.loc[train_test['10']['train_indxs'], 'sp_pavgprice']
print(X_train.shape)

X_test = df_modelAVG_label.loc[train_test['10']['test_indxs'], select_vars]
y_test = df_modelAVG_label.loc[train_test['10']['test_indxs'],'sp_pavgprice']
print(X_test.shape)

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
train_scores = []
test_scores = []


for x in test_dates:

  start = time.time()

  test_d = '2023-06-' + x
  test_days.append(test_d)
  print('_'*35)
  print(f'Model XGBoost (Test Day {x})')

  X_train = df_modelAVG_label.loc[train_test[x]['train_indxs'], select_vars]
  y_train = df_modelAVG_label.loc[train_test[x]['train_indxs'], 'sp_pavgprice']

  X_test = df_modelAVG_label.loc[train_test[x]['test_indxs'], select_vars]
  y_test = df_modelAVG_label.loc[train_test[x]['test_indxs'],'sp_pavgprice']


  RF_bid.fit(X_train, y_train)


  size_train = y_train.shape[0]
  sumresid_train = sum(y_train - RF_bid.predict(X_train))
  stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
  r2_train = r2_score(y_train, RF_bid.predict(X_train))
  rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
  train_scores.append(rmse_train)
  print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
  print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

  size_test = y_test.shape[0]
  sumresid_test = sum(y_test - RF_bid.predict(X_test))
  stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
  r2_test = r2_score(y_test, RF_bid.predict(X_test))
  rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
  test_scores.append(rmse_test)
  print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
  print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


  model_imp = 'imp_' + x
  #model_importances[model_imp] = RF_bid.feature_importances_

  end = time.time()
  print(timedelta(minutes=end-start))
  #print('\n')

model_scores = {'test_days':test_days,'train_scores':train_scores,'test_scores':test_scores}


In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum
0,marketIndexPrice,NaN
1,marketIndexVolume,NaN
2,LOC Center LAT,NaN
3,Holiday,NaN
4,Weekday_SIN,NaN
5,Weekday_COS,NaN
6,Month_SIN,NaN
7,Month_COS,NaN
8,Day_SIN,NaN
9,Day_COS,NaN


In [ ]:
model_imp_df.head(20)

,feature,imp_sum
0,marketIndexPrice,NaN
1,marketIndexVolume,NaN
2,LOC Center LAT,NaN
3,Holiday,NaN
4,Weekday_SIN,NaN
5,Weekday_COS,NaN
6,Month_SIN,NaN
7,Month_COS,NaN
8,Day_SIN,NaN
9,Day_COS,NaN


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum
19,NatGas_price,NaN
20,Coal_price,NaN
21,BrentOil_price,NaN
22,sp3_6_meanvolume,NaN
23,sp3_6_totalvolume,NaN
24,sp6_12_meanvolume,NaN
25,sp6_12_totalvolume,NaN
26,sp12_24_pavgprice,NaN
27,sp12_24_meanvolume,NaN
28,sp12_24_totalvolume,NaN


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo BID (Predicción por Mañana/Tarde)

In [ ]:
recordtype = 'BID'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
test_dates = ['01','06','10','15','20','24','29']
test_periods = ['Mañana SP1-24','Tarde SP25-48']

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}


  test_sp = 'Mañana SP1-24'

  train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_indxs = list(train_prev.index.values)
  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<=24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}



  test_sp = 'Tarde SP25-48'

  train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==df_modelAVG_label)&(df_modelAVG_label['settlementPeriod']<=24)].copy()
  train = pd.concat([train_prev, train_day])
  train_indxs = list(train.index.values)

  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']>24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

In [ ]:
x = '2023-06-01'
print(train_test[x].keys())

In [ ]:
x = '2023-06-01'
y = 'Mañana SP1-24'
#y = 'Tarde SP25-48'

In [ ]:
train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'],:]
print(train.shape)
train

In [ ]:
df_modelAVG_label.loc[train_test[x][y]['test_indxs'],:]
print(test.shape)
test

In [ ]:
train_test[x][y]['train_indxs'][-1]

In [ ]:
train_test[x][y]['test_indxs'][-1]

In [ ]:
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

for x in test_dates:
  for y in test_periods:

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')
    print(len(train_test[x][y]['train_indxs']))
    print(len(train_test[x][y]['test_indxs']))

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation']
'''

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

In [ ]:
train_test.keys()

In [ ]:
train_test['2023-06-01'].keys()

In [ ]:
train_test['2023-06-01']['Mañana SP1-24'].keys()

In [ ]:
X_train = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], select_vars]
y_train = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], 'sp_pavgprice']
print(X_train.shape)

X_test = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'], select_vars]
y_test = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'],'sp_pavgprice']
print(X_test.shape)

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
test_sps = []
train_scores = []
test_scores = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    sumresid_train = sum(y_train - RF_bid.predict(X_train))
    stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
    r2_train = r2_score(y_train, RF_bid.predict(X_train))
    rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
    train_scores.append(rmse_train)
    print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
    print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

    size_test = y_test.shape[0]
    sumresid_test = sum(y_test - RF_bid.predict(X_test))
    stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
    r2_test = r2_score(y_test, RF_bid.predict(X_test))
    rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
    test_scores.append(rmse_test)
    print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
    print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


    if y == 'Mañana SP1-24':
      model_imp = 'imp_' + x + 'M'
    else:
      model_imp = 'imp_' + x + 'T'
    model_importances[model_imp] = RF_bid.feature_importances_

    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_days':test_days,'test_sps':test_sps,'train_scores':train_scores,'test_scores':test_scores}


In [ ]:
#pd.DataFrame(model_scores)

In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

In [ ]:
model_imp_df.head(20)

In [ ]:
model_imp_df.tail(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo BID (Predicción por SP)

In [129]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [130]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-03', '2023-06-05', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-13', '2023-06-15', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-23', '2023-06-25', '2023-06-27', '2023-06-29'])

In [131]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

229670
5
229700
6
229743
6
229782
7
229808
7
229846
7
229884
8
229908
10
229974
5
230277
5
230312
8
230343
4
230374
4
230389
4
230409
5
230442
11
230466
5
230509
2
230680
5
230705
6
230733
3
230748
1
230754
4
230782
7
230825
9
230847
8
230914
3
231130
7
231166
5
231206
5
231227
0
231227
4
231254
5
231291
3
231297
3
231340
8
231662
6
231683
2
231716
4
231750
5
231759
3
231785
3
231816
6
231834
4
231878
5
232090
1
232097
0
232122
3
232149
10
232165
2
232199
4
232218
9
232232
9
232269
4
232607
9
232639
7
232682
12
232729
7
232749
8
232785
4
232815
5
232835
7
232898
5
233161
2
233172
1
233217
7
233272
9
233293
2
233312
8
233347
3
233350
1
233393
6
233675
2
233699
3
233731
5
233768
6
233789
6
233830
7
233871
4
233886
5
233925
5
234212
4
234244
5
234294
8
234331
5
234346
4
234374
4
234402
6
234428
11
234518
9
234815
4
234836
7
234879
7
234921
3
234932
7
234969
4
234995
5
235021
5
235063
9
235357
8
235398
7
235441
5
235469
7
235487
5
235519
1
235542
13
235570
5
235614
7
235872
7
235904
4
2359

In [132]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['recordtype_label', 'Year_ordin', 'settPeriod_COS','Day_SIN',
                 'Holiday', 'Month_ordin', 'Day_ordin',
                 'sp6_12_totalvolume', 'marketIndexPrice', 'marketIndexVolume',
                 'NatGas_price', 'SystemTotalLoad_Consumption', 'sp3_6_totalvolume', 'BrentOil_price', 'WindOffshore_Generation',
                 'DAMargin', 'sp6_12_meanvolume', 'GenerationForecast_Production', 'settlementDate_ordin', 'BMU GSP Group Name_label',
                 'DATF_Demand', 'settlementPeriod_ordin', 'DAImbalance', 'Weekday_COS', 'Solar_Generation', 'Weekday_SIN', 'Day_COS'
                 ]


'''

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['recordtype_label', 'Year_ordin', 'settPeriod_COS','Day_SIN',\n                 'Holiday', 'Month_ordin', 'Day_ordin',\n                 'sp6_12_totalvolume', 'marketIndexPrice', 'marketIndexVolume',\n                 'NatGas_price', 'SystemTotalLoad_Consumption', 'sp3_6_totalvolume', 'BrentOil_price', 'WindOffshore_Generation',\n                 'DAMargin', 'sp6_12_meanvolume', 'GenerationForecast_Production', 'settlementDate_ordin', 'BMU GSP Group Name_label',\n                 'DATF_Demand', 'settlementPeriod_ordin', 'DAImbalance', 'Weekday_COS', 'Solar_Generation', 'Weekday_SIN', 'Day_COS'\n                 ]\n\n\n"

In [133]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [134]:
len(select_vars)

40

In [135]:
select_vars[:5]

['LOC Center LAT', 'LOC Center LONG', 'Weekend', 'Holiday', 'Weekday_SIN']

In [136]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [137]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=229670 / RMSE=28.270964670503083 / MAE=15.17572541734193 / MeanReal: 73.17324911328387 / MeanPred: 73.17400360107422
              Resid=-173.91466418959314 / AbsResidMean=15.17572541734193 / ResidMean=-0.000757237184615822 / StdResidMean=0.0001230938505697818
Test scores: N=5 / RMSE=6.880697998691888 / MAE=5.75498510939529 / MeanReal: 30.074272473083436 / MeanPred: 24.402578353881836
             Resid=28.358468688659386 / AbsResidMean=5.75498510939529 / ResidMean=5.671693737731877 / StdResidMean=0.7791250244877408
0:23:31.462154
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=229700 / RMSE=28.627122586985266 / MAE=15.222758227586349 / MeanReal: 73.16778496979659 / MeanPred: 73.17095184326172
              Resid=-726.9180372562661 / AbsResidMean=15.222758227586349 / ResidMean=-0.0031646409980735793 / StdResidMean=0.0001246

In [138]:
pd.DataFrame(model_scores)

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,229670,28.270965,15.175725,73.173249,73.173249,-173.914664,15.175725,-0.000757,0.000123,5,6.880698,5.754985,30.074272,24.402578,28.358469,5.754985,5.671694,0.779125
1,2023-06-01,10,229700,28.627123,15.222758,73.167785,73.167785,-726.918037,15.222758,-0.003165,0.000125,6,8.128014,5.087288,32.191245,27.103958,30.523730,5.087288,5.087288,1.056516
2,2023-06-01,16,229743,28.639427,15.179462,73.160284,73.160284,-44.961781,15.179462,-0.000196,0.000125,6,8.426874,4.633058,30.040695,34.673756,-27.798350,4.633058,-4.633058,1.173160
3,2023-06-01,22,229782,29.138776,15.130359,73.153514,73.153514,-21.964394,15.130359,-0.000096,0.000127,7,7.931564,6.423569,29.932998,30.429596,-3.476172,6.423569,-0.496596,1.130857
4,2023-06-01,25,229808,28.980830,15.269839,73.149549,73.149549,-537.301112,15.269839,-0.002338,0.000126,7,12.539711,8.199045,29.423439,32.449139,-21.179916,8.199045,-3.025702,1.738457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,237099,28.203668,15.095280,72.447233,72.447233,-425.268784,15.095280,-0.001794,0.000119,8,4.277878,3.109423,53.973159,53.467968,4.041523,3.109423,0.505190,0.530993
131,2023-06-29,31,237161,28.654007,15.011854,72.442964,72.442964,-704.606855,15.011854,-0.002971,0.000121,9,12.870862,7.663990,57.624310,50.502861,64.093044,7.663990,7.121449,1.191244
132,2023-06-29,37,237216,29.011845,15.104719,72.439203,72.439203,-111.310234,15.104719,-0.000469,0.000122,7,4.299421,2.225756,53.403294,55.043068,-11.478419,2.225756,-1.639774,0.567777
133,2023-06-29,40,237231,28.851811,15.133274,72.437787,72.437787,-315.970317,15.133274,-0.001332,0.000122,6,6.860274,4.274325,52.449252,52.990032,-3.244688,4.274325,-0.540781,1.139821


In [141]:
model_scores_df = pd.DataFrame(model_scores)

In [159]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID SP PAVG. PRICE MODEL - SP (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [160]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,229670,28.270965,15.175725,73.173249,73.173249,-173.914664,15.175725,-0.000757,0.000123,5,6.880698,5.754985,30.074272,24.402578,28.358469,5.754985,5.671694,0.779125
1,2023-06-01,10,229700,28.627123,15.222758,73.167785,73.167785,-726.918037,15.222758,-0.003165,0.000125,6,8.128014,5.087288,32.191245,27.103958,30.523730,5.087288,5.087288,1.056516
2,2023-06-01,16,229743,28.639427,15.179462,73.160284,73.160284,-44.961781,15.179462,-0.000196,0.000125,6,8.426874,4.633058,30.040695,34.673756,-27.798350,4.633058,-4.633058,1.173160
3,2023-06-01,22,229782,29.138776,15.130359,73.153514,73.153514,-21.964394,15.130359,-0.000096,0.000127,7,7.931564,6.423569,29.932998,30.429596,-3.476172,6.423569,-0.496596,1.130857
4,2023-06-01,25,229808,28.980830,15.269839,73.149549,73.149549,-537.301112,15.269839,-0.002338,0.000126,7,12.539711,8.199045,29.423439,32.449139,-21.179916,8.199045,-3.025702,1.738457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,237099,28.203668,15.095280,72.447233,72.447233,-425.268784,15.095280,-0.001794,0.000119,8,4.277878,3.109423,53.973159,53.467968,4.041523,3.109423,0.505190,0.530993
131,2023-06-29,31,237161,28.654007,15.011854,72.442964,72.442964,-704.606855,15.011854,-0.002971,0.000121,9,12.870862,7.663990,57.624310,50.502861,64.093044,7.663990,7.121449,1.191244
132,2023-06-29,37,237216,29.011845,15.104719,72.439203,72.439203,-111.310234,15.104719,-0.000469,0.000122,7,4.299421,2.225756,53.403294,55.043068,-11.478419,2.225756,-1.639774,0.567777
133,2023-06-29,40,237231,28.851811,15.133274,72.437787,72.437787,-315.970317,15.133274,-0.001332,0.000122,6,6.860274,4.274325,52.449252,52.990032,-3.244688,4.274325,-0.540781,1.139821


In [161]:
model_scores_df['rmse_test'].mean()

32.54483074279318

In [162]:
model_scores_df['mae_test'].mean()

26.844546421310607

In [158]:
model_scores_df['meanreal_test'].mean()

45.28154068887807

In [164]:
model_scores_df.sort_values(by=['rmse_test'], ascending=False).head(10)

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
70,2023-06-15,40,233350,27.934897,15.006131,72.771212,72.771212,-117.560320,15.006131,-0.000504,0.000120,1,1308.659424,1308.659424,70.000000,1378.659424,-1308.659424,1308.659424,-1308.659424,0.000000
53,2023-06-11,48,232269,28.318168,15.075896,72.825193,72.825193,-353.909779,15.075896,-0.001524,0.000122,4,1204.431629,905.340349,53.493750,958.834106,-3621.361395,905.340349,-905.340349,198.591037
108,2023-06-25,4,235872,28.122633,14.857820,72.674834,72.674834,20.193618,14.857820,0.000086,0.000119,7,103.630485,74.172137,-51.233803,21.730200,-510.748021,74.172137,-72.964003,10.512900
71,2023-06-15,48,233393,27.600772,15.033294,72.777939,72.777939,-289.281265,15.033294,-0.001239,0.000118,6,80.334876,70.670797,72.470157,1.799360,424.024785,70.670797,70.670797,6.366864
118,2023-06-27,10,236401,28.342382,15.057258,72.528454,72.528454,-123.585016,15.057258,-0.000523,0.000120,4,62.896623,46.741907,15.376451,56.200714,-163.297043,46.741907,-40.824261,11.961827
107,2023-06-23,48,235614,28.610189,15.007626,72.709888,72.709888,-154.080976,15.007626,-0.000654,0.000121,7,57.253218,34.059723,65.824504,63.885612,13.572244,34.059723,1.938892,8.174340
62,2023-06-13,48,232898,28.164526,15.024881,72.789703,72.789703,-1062.639921,15.024881,-0.004563,0.000121,5,47.218886,29.957099,60.421929,71.325798,-54.519334,29.957099,-10.903867,9.188534
106,2023-06-23,40,235570,27.961606,14.967420,72.716489,72.716489,-593.312220,14.967420,-0.002519,0.000119,5,45.698709,31.896542,26.747300,52.277557,-127.651288,31.896542,-25.530258,7.580443
112,2023-06-25,25,235964,28.709115,15.024000,72.631260,72.631260,-546.431557,15.024000,-0.002316,0.000122,3,45.030603,38.037001,36.732410,5.049730,95.048044,38.037001,31.682681,10.666474
45,2023-06-11,4,232090,27.820741,15.013288,72.788187,72.788187,-586.448903,15.013288,-0.002527,0.000120,1,39.018793,39.018793,21.014805,-18.003988,39.018793,39.018793,39.018793,0.000000


In [147]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [148]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID SP PAVG. PRICE MODEL - SP (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [149]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.206893,0.208967,0.213634,0.239862,0.220004,0.234379,0.192302,0.183247,0.217735,0.197524,0.178915,0.197474,0.199884,0.196794,0.215098,0.207578
1,Month_COS,0.197311,0.178428,0.198362,0.168396,0.215789,0.187992,0.189373,0.187850,0.150374,0.163211,0.179695,0.246218,0.213019,0.302373,0.231464,0.147123
2,Month_SIN,0.080203,0.138220,0.096641,0.130237,0.140929,0.132060,0.111687,0.088706,0.096306,0.109777,0.112374,0.010076,0.021562,0.002123,0.011003,0.001342
3,Weekend,0.061802,0.032107,0.119912,0.035293,0.030020,0.025216,0.021649,0.094901,0.062057,0.120711,0.079179,0.049216,0.067528,0.068494,0.050940,0.069805
4,LOC Center LONG,0.055843,0.063348,0.050776,0.065543,0.052059,0.054884,0.033941,0.042596,0.057274,0.054313,0.040738,0.056177,0.066042,0.052171,0.080010,0.067779
5,DAID_spnDemand,0.047220,0.052004,0.049220,0.048178,0.047726,0.048000,0.056232,0.044325,0.055717,0.055128,0.042921,0.048301,0.037720,0.054312,0.029135,0.039375
6,Day_SIN,0.036124,0.015544,0.003988,0.002940,0.002779,0.002542,0.058169,0.039922,0.002722,0.007557,0.119274,0.061291,0.045155,0.060434,0.060635,0.058902
7,settPeriod_SIN,0.036020,0.026860,0.026085,0.046987,0.046370,0.041826,0.051311,0.051329,0.040908,0.044822,0.027788,0.024837,0.019018,0.033973,0.025282,0.032894
8,DANF_Demand,0.030730,0.038128,0.025777,0.039043,0.032050,0.030660,0.034307,0.025179,0.047324,0.028848,0.024631,0.036322,0.042507,0.015640,0.017633,0.022899
9,LOC Center LAT,0.024149,0.028157,0.029622,0.035154,0.032679,0.033951,0.021401,0.019289,0.029541,0.021855,0.020780,0.017184,0.017254,0.020065,0.026002,0.009301


In [150]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.206893,0.208967,0.213634,0.239862,0.220004,0.234379,0.192302,0.183247,0.217735,0.197524,0.178915,0.197474,0.199884,0.196794,0.215098,0.207578
1,Month_COS,0.197311,0.178428,0.198362,0.168396,0.215789,0.187992,0.189373,0.187850,0.150374,0.163211,0.179695,0.246218,0.213019,0.302373,0.231464,0.147123
2,Month_SIN,0.080203,0.138220,0.096641,0.130237,0.140929,0.132060,0.111687,0.088706,0.096306,0.109777,0.112374,0.010076,0.021562,0.002123,0.011003,0.001342
3,Weekend,0.061802,0.032107,0.119912,0.035293,0.030020,0.025216,0.021649,0.094901,0.062057,0.120711,0.079179,0.049216,0.067528,0.068494,0.050940,0.069805
4,LOC Center LONG,0.055843,0.063348,0.050776,0.065543,0.052059,0.054884,0.033941,0.042596,0.057274,0.054313,0.040738,0.056177,0.066042,0.052171,0.080010,0.067779
5,DAID_spnDemand,0.047220,0.052004,0.049220,0.048178,0.047726,0.048000,0.056232,0.044325,0.055717,0.055128,0.042921,0.048301,0.037720,0.054312,0.029135,0.039375
6,Day_SIN,0.036124,0.015544,0.003988,0.002940,0.002779,0.002542,0.058169,0.039922,0.002722,0.007557,0.119274,0.061291,0.045155,0.060434,0.060635,0.058902
7,settPeriod_SIN,0.036020,0.026860,0.026085,0.046987,0.046370,0.041826,0.051311,0.051329,0.040908,0.044822,0.027788,0.024837,0.019018,0.033973,0.025282,0.032894
8,DANF_Demand,0.030730,0.038128,0.025777,0.039043,0.032050,0.030660,0.034307,0.025179,0.047324,0.028848,0.024631,0.036322,0.042507,0.015640,0.017633,0.022899
9,LOC Center LAT,0.024149,0.028157,0.029622,0.035154,0.032679,0.033951,0.021401,0.019289,0.029541,0.021855,0.020780,0.017184,0.017254,0.020065,0.026002,0.009301


In [151]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.206893,0.208967,0.213634,0.239862,0.220004,0.234379,0.192302,0.183247,0.217735,0.197524,0.178915,0.197474,0.199884,0.196794,0.215098,0.207578
1,Month_COS,0.197311,0.178428,0.198362,0.168396,0.215789,0.187992,0.189373,0.187850,0.150374,0.163211,0.179695,0.246218,0.213019,0.302373,0.231464,0.147123
2,Month_SIN,0.080203,0.138220,0.096641,0.130237,0.140929,0.132060,0.111687,0.088706,0.096306,0.109777,0.112374,0.010076,0.021562,0.002123,0.011003,0.001342
3,Weekend,0.061802,0.032107,0.119912,0.035293,0.030020,0.025216,0.021649,0.094901,0.062057,0.120711,0.079179,0.049216,0.067528,0.068494,0.050940,0.069805
4,LOC Center LONG,0.055843,0.063348,0.050776,0.065543,0.052059,0.054884,0.033941,0.042596,0.057274,0.054313,0.040738,0.056177,0.066042,0.052171,0.080010,0.067779
5,DAID_spnDemand,0.047220,0.052004,0.049220,0.048178,0.047726,0.048000,0.056232,0.044325,0.055717,0.055128,0.042921,0.048301,0.037720,0.054312,0.029135,0.039375
6,Day_SIN,0.036124,0.015544,0.003988,0.002940,0.002779,0.002542,0.058169,0.039922,0.002722,0.007557,0.119274,0.061291,0.045155,0.060434,0.060635,0.058902
7,settPeriod_SIN,0.036020,0.026860,0.026085,0.046987,0.046370,0.041826,0.051311,0.051329,0.040908,0.044822,0.027788,0.024837,0.019018,0.033973,0.025282,0.032894
8,DANF_Demand,0.030730,0.038128,0.025777,0.039043,0.032050,0.030660,0.034307,0.025179,0.047324,0.028848,0.024631,0.036322,0.042507,0.015640,0.017633,0.022899
9,LOC Center LAT,0.024149,0.028157,0.029622,0.035154,0.032679,0.033951,0.021401,0.019289,0.029541,0.021855,0.020780,0.017184,0.017254,0.020065,0.026002,0.009301


In [152]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
20,Weekday_SIN,0.008550,0.005684,0.001679,0.007286,0.007734,0.005691,0.012197,0.004734,0.012320,0.006132,0.002779,0.018549,0.004305,0.013240,0.004859,0.021064
21,DAMargin,0.008298,0.011897,0.004700,0.006234,0.007470,0.009178,0.008141,0.008020,0.008538,0.008335,0.013744,0.009079,0.007876,0.005825,0.006807,0.008625
22,WindOffshore_Generation,0.007666,0.007654,0.005348,0.009334,0.013287,0.010234,0.008565,0.004959,0.009630,0.006237,0.005282,0.008641,0.003541,0.005354,0.010593,0.006331
23,sp3_6_totalvolume,0.007216,0.007651,0.005928,0.006803,0.006164,0.007204,0.006698,0.006471,0.007308,0.005938,0.005167,0.006350,0.007954,0.013193,0.007412,0.008005
24,BrentOil_price,0.006696,0.014645,0.010160,0.011104,0.008405,0.005474,0.006799,0.004189,0.009063,0.006452,0.004693,0.003204,0.002938,0.002327,0.003570,0.007421
25,Solar_Generation,0.004968,0.004490,0.013089,0.004001,0.001785,0.010489,0.004354,0.006173,0.005128,0.001677,0.001319,0.003469,0.009103,0.001226,0.004012,0.004201
26,GenerationForecast_Production,0.004601,0.006926,0.005902,0.005425,0.004421,0.006656,0.002301,0.004016,0.003554,0.007659,0.002090,0.002284,0.002460,0.008254,0.003465,0.003601
27,DAImbalance,0.004364,0.001619,0.001484,0.002157,0.002910,0.001725,0.001413,0.009468,0.016985,0.002508,0.001026,0.003544,0.009337,0.004669,0.002531,0.004080
28,BMU GSP Group Name_label,0.004263,0.004155,0.004012,0.004642,0.002283,0.006235,0.004511,0.002326,0.003691,0.003072,0.002320,0.004295,0.006818,0.004425,0.002978,0.008182
29,settlementDate_ordin,0.004075,0.003002,0.002822,0.003337,0.004343,0.003399,0.009607,0.005716,0.004880,0.003642,0.003660,0.003479,0.003611,0.003127,0.003288,0.003206


In [153]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [154]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

In [ ]:
RF_bid.predict(X_test)

array([57.202824, 58.843002, 48.156994, -8.666964, 54.652542, 68.950905,
       39.413742, 57.108395, 55.45231 , 54.971672], dtype=float32)

In [ ]:
df_modelAVG[(df_modelAVG['recordtype']=='BID')&(df_modelAVG['settlementDate']=='2023-06-10')&(df_modelAVG['settlementPeriod']==33)]

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,LOC Center LAT,LOC Center LONG,Weekend,Holiday,Year,...,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
179315,BID,2023-06-10,33,North Western England,60.00000,54.127464,-2.769975,1.0,0.0,2023,...,3807.983,32.05,139.65,74.79,29.3250,-0.050,-0.200,9.456657,-0.052429,-0.367
467610,BID,2023-06-10,33,Yorkshire,42.10985,53.670497,-1.071275,1.0,0.0,2023,...,3807.983,32.05,139.65,74.79,56.7375,-0.113,-0.452,11.891348,-4.664500,-38.316


# OPTIMIZACIÓN INICIAL DE MODELOS (librería "Optuna")

### Carga Librerías

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
%matplotlib inline

In [ ]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Busquemos los mejor hiperparámetros con Optuna

Para ello hay que envolver todo el modelo con la función objective

### XGBOOST (BID MODEL - SP)

In [ ]:
from matplotlib.cbook import print_cycles
unselect_vars = ['recordtype_label', 'Year_ordin']

select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

print(len(select_vars))

73


In [ ]:
recordtype = 'BID'

#test_dates = ['01','06','10','15','20','24','29']
test_dates = ['10','15','29']
#test_speriods = [14,24,34,42]
test_speriods = [14,34]

train_test = {}
cv_list = []

for x in test_dates:

  test_day = '2023-05-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}
    cv_list.append((train_indxs,test_indxs))

In [ ]:

def objective(trial, X, y, cv, scoring):
    rf_params = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=50, high=200, log=False),
        "max_depth":trial.suggest_int(name="max_depth", low=5, high=10, log=False),
        "colsample_bytree":trial.suggest_float(name="colsample_bytree", low=0.3, high=1, log=False),
        "eta":trial.suggest_float(name="eta", low=0.1, high=0.4, log=False),
        "n_jobs": -1,
        "random_state": 0
        }

    steps = [('predictor', XGBRegressor(**rf_params))]

    pipe = Pipeline(steps)
    scores = cross_val_score(pipe, X, y, cv=cv,  scoring=scoring, n_jobs=-1)

    return scores.mean()


In [ ]:
#sklearn.metrics.get_scorer_names()

In [ ]:
%%time
from sklearn.model_selection import cross_val_score


#from sklearn.metrics import r2_score
#my_scorer = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
#sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))

# Create study that maximizes
study_XG1 = optuna.create_study(direction="maximize")


# Pass additional arguments inside another function
X_train = df_model_label[select_vars]
y_train = df_model_label['acceptedprice']

func = lambda trial: objective(trial, X_train, y_train, cv=cv_list,  scoring='neg_mean_squared_error')   #cv=kf


# Start optimizing with 100 trials
study_XG1.optimize(func, n_trials=25)

[I 2023-07-17 13:27:25,223] A new study created in memory with name: no-name-47240e0e-cc9b-42ab-9316-ace2422734b4
[I 2023-07-17 13:29:45,558] Trial 0 finished with value: -282.77589628475783 and parameters: {'n_estimators': 66, 'max_depth': 5, 'colsample_bytree': 0.334487797039703, 'eta': 0.3401202789441894}. Best is trial 0 with value: -282.77589628475783.
[I 2023-07-17 13:52:06,937] Trial 1 finished with value: -204.83128015784482 and parameters: {'n_estimators': 200, 'max_depth': 9, 'colsample_bytree': 0.7092114719682486, 'eta': 0.14935032440192256}. Best is trial 1 with value: -204.83128015784482.
[I 2023-07-17 13:58:54,716] Trial 2 finished with value: -220.19015177213615 and parameters: {'n_estimators': 93, 'max_depth': 8, 'colsample_bytree': 0.4785003988112828, 'eta': 0.159975058116462}. Best is trial 1 with value: -204.83128015784482.
[I 2023-07-17 14:02:36,919] Trial 3 finished with value: -358.2506233409368 and parameters: {'n_estimators': 111, 'max_depth': 5, 'colsample_bytr

CPU times: user 11min 1s, sys: 1min 6s, total: 12min 7s
Wall time: 6h 8min 9s


Visualizando el estudio

La verdad es que para obtener el mejor n_estimators tampoco era necesario usar optuna: un gridsearch corriente hubiera bastando...
Pero lo que es muy interesante es la visualización del estudio.

In [ ]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study_XG1)

In [ ]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study_XG1)

In [ ]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study_XG1)

In [ ]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study_XG1)

In [ ]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study_XG1)

Vamos a fija el número de árboles lo más bajo posible dentro de los mejores resultados. De este modo nos curamos del OF y no son demasiados para no reventar el tiempo de cálculo.

In [ ]:
''' XGBOOST



'''

' XGBOOST\n\n\n\n'

In [ ]:
# To get the dictionary of parameter name and parameter values:
print("Return a dictionary of parameter name and parameter values:",study_XG1.best_params)

# To get the best observed value of the objective function:
print("Return the best observed value of the objective function:",study_XG1.best_value)

# To get the best trial:
print("Return the best trial:",study_XG1.best_trial)

# To get all trials:
#print("Return all the trials:", study.trials)

Return a dictionary of parameter name and parameter values: {'n_estimators': 151, 'max_depth': 9, 'colsample_bytree': 0.8012678112481448, 'eta': 0.3203346608008277}
Return the best observed value of the objective function: -139.38123127195743
Return the best trial: FrozenTrial(number=20, state=TrialState.COMPLETE, values=[-139.38123127195743], datetime_start=datetime.datetime(2023, 7, 17, 18, 4, 1, 374838), datetime_complete=datetime.datetime(2023, 7, 17, 18, 22, 8, 146717), params={'n_estimators': 151, 'max_depth': 9, 'colsample_bytree': 0.8012678112481448, 'eta': 0.3203346608008277}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=False, low=50, step=1), 'max_depth': IntDistribution(high=10, log=False, low=5, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.3, step=None), 'eta': FloatDistribution(high=0.4, log=False, low=0.1, step=None)}, trial_id=20, value=None)


### XGBOOST (BID MODEL - SP PAVG. PRICE MODEL)

In [165]:
from matplotlib.cbook import print_cycles
unselect_vars = ['recordtype_label', 'Year_ordin']

select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

print(len(select_vars))

40


In [166]:
recordtype = 'BID'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
test_dates = ['10','15','29']
#test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
test_speriods = [14,34]

train_test = {}
cv_list = []

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}
    cv_list.append((train_indxs,test_indxs))

In [176]:

def objective(trial, X, y, cv, scoring):
    rf_params = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=20, high=150, log=False),
        "max_depth":trial.suggest_int(name="max_depth", low=2, high=6, log=False),
        "colsample_bytree":trial.suggest_float(name="colsample_bytree", low=0, high=0.5, log=False),
        "eta":trial.suggest_float(name="eta", low=0.2, high=0.35, log=False),
        "n_jobs": -1,
        "random_state": 0
        }

    steps = [('predictor', XGBRegressor(**rf_params))]

    pipe = Pipeline(steps)
    scores = cross_val_score(pipe, X, y, cv=cv,  scoring=scoring, n_jobs=-1)

    return scores.mean()


In [177]:
#sklearn.metrics.get_scorer_names()

In [178]:
%%time
from sklearn.model_selection import cross_val_score


#from sklearn.metrics import r2_score
#my_scorer = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
#sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))

# Create study that minimizes
study_XG2 = optuna.create_study(direction="maximize")


# Pass additional arguments inside another function
X_train = df_modelAVG_label[select_vars]
y_train = df_modelAVG_label['sp_pavgprice']

func = lambda trial: objective(trial, X_train, y_train, cv=cv_list,  scoring='neg_mean_squared_error')   #cv=kf


# Start optimizing with 100 trials
study_XG2.optimize(func, n_trials=25)

[I 2023-07-23 14:15:24,243] A new study created in memory with name: no-name-da2c42f7-bb3e-4fd6-ae7b-11f7d66ce4b0
[I 2023-07-23 14:15:35,722] Trial 0 finished with value: -1462.3660655556025 and parameters: {'n_estimators': 26, 'max_depth': 4, 'colsample_bytree': 0.08275797871933988, 'eta': 0.2835759107145698}. Best is trial 0 with value: -1462.3660655556025.
[I 2023-07-23 14:15:54,787] Trial 1 finished with value: -1823.2358298979605 and parameters: {'n_estimators': 73, 'max_depth': 2, 'colsample_bytree': 0.34849182934805883, 'eta': 0.2810719536186657}. Best is trial 0 with value: -1462.3660655556025.
[I 2023-07-23 14:16:52,818] Trial 2 finished with value: -2175.1991932097903 and parameters: {'n_estimators': 121, 'max_depth': 5, 'colsample_bytree': 0.3432073690830865, 'eta': 0.33469028928274996}. Best is trial 0 with value: -1462.3660655556025.
[I 2023-07-23 14:17:13,988] Trial 3 finished with value: -1774.6467514537833 and parameters: {'n_estimators': 89, 'max_depth': 4, 'colsample_

CPU times: user 2min 17s, sys: 5.83 s, total: 2min 23s
Wall time: 8min 38s


Visualizando el estudio

La verdad es que para obtener el mejor n_estimators tampoco era necesario usar optuna: un gridsearch corriente hubiera bastando...
Pero lo que es muy interesante es la visualización del estudio.

In [179]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study_XG2)

In [180]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study_XG2)

In [181]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study_XG2)

In [182]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study_XG2)

In [183]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study_XG2)

Vamos a fija el número de árboles lo más bajo posible dentro de los mejores resultados. De este modo nos curamos del OF y no son demasiados para no reventar el tiempo de cálculo.

In [184]:
# To get the dictionary of parameter name and parameter values:
print("Return a dictionary of parameter name and parameter values:",study_XG2.best_params)

# To get the best observed value of the objective function:
print("Return the best observed value of the objective function:",study_XG2.best_value)

# To get the best trial:
print("Return the best trial:",study_XG2.best_trial)

# To get all trials:
#print("Return all the trials:", study.trials)

Return a dictionary of parameter name and parameter values: {'n_estimators': 55, 'max_depth': 2, 'colsample_bytree': 0.06685323090676709, 'eta': 0.2893124969719224}
Return the best observed value of the objective function: -1189.0514719952419
Return the best trial: FrozenTrial(number=4, state=TrialState.COMPLETE, values=[-1189.0514719952419], datetime_start=datetime.datetime(2023, 7, 23, 14, 17, 13, 990840), datetime_complete=datetime.datetime(2023, 7, 23, 14, 17, 24, 126970), params={'n_estimators': 55, 'max_depth': 2, 'colsample_bytree': 0.06685323090676709, 'eta': 0.2893124969719224}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=150, log=False, low=20, step=1), 'max_depth': IntDistribution(high=6, log=False, low=2, step=1), 'colsample_bytree': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'eta': FloatDistribution(high=0.35, log=False, low=0.2, step=None)}, trial_id=4, value=None)


# MODELOS AJUSTADOS (tras optimización)

**XGBOOST (BID/OFFER MODEL)**

Return a dictionary of parameter name and parameter values: {'n_estimators': 151, 'max_depth': 9, 'colsample_bytree': 0.8012678112481448, 'eta': 0.3203346608008277}

Return the best observed value of the objective function: -139.38123127195743

</br>

~~Return a dictionary of parameter name and parameter values: {'n_estimators': 80, 'max_depth': 7, 'colsample_bytree': 0.5764929525944595, 'eta': 0.23497115410695124}~~

~~Return the best observed value of the objective function: -177.23714263467116~~


XGBOOST (SP PAVG. PRICE MODEL)

Return a dictionary of parameter name and parameter values: {'n_estimators': 55, 'max_depth': 2, 'colsample_bytree': 0.06685323090676709, 'eta': 0.2893124969719224}

Return the best observed value of the objective function: -1189.0514719952419

### XGBOOST (BID/OFFER MODEL) - Modelo BID (Predicción por SP)

In [ ]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

725769
10
725830
10
725912
9
725965
10
726005
10
726066
12
726138
17
726182
25
726337
8
726923
11
727008
20
727095
13
727178
14
727213
8
727246
6
727316
23
727363
7
727421
3
727731
8
727779
14
727859
6
727898
3
727912
9
727974
19
728057
15
728094
19
728277
6
728630
9
728681
7
728763
9
728794
0
728794
6
728847
10
728912
4
728923
4
729003
14
729655
11
729699
6
729769
9
729836
10
729857
6
729895
4
729945
10
729970
5
730047
10
730608
4
730620
0
730665
10
730725
15
730754
5
730817
6
730839
13
730859
13
730914
4
731602
17
731668
12
731749
38
731874
17
731919
15
731981
5
732035
11
732068
10
732177
8
732564
5
732584
1
732670
12
732769
14
732800
3
732833
15
732889
5
732894
1
732964
9
733421
2
733449
4
733502
7
733564
10
733597
10
733673
9
733739
10
733769
7
733840
7
734360
7
734423
12
734546
17
734616
7
734637
6
734701
20
734798
21
734885
38
735132
17
735760
7
735806
15
735913
12
735992
5
736012
12
736090
11
736155
12
736214
7
736293
12
736822
14
736898
13
736993
11
737031
15
737070
10
737159
2

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

len(select_vars)

74

In [ ]:
'''
BEST HYPERPARAMETERS (OPTUNA)
{'n_estimators': 151, 'max_depth': 9, 'colsample_bytree': 0.8012678112481448, 'eta': 0.3203346608008277}
{'n_estimators': 80, 'max_depth': 7, 'colsample_bytree': 0.5764929525944595, 'eta': 0.23497115410695124}

'''

"\nBEST HYPERPARAMETERS (OPTUNA)\n{'n_estimators': 151, 'max_depth': 9, 'colsample_bytree': 0.8012678112481448, 'eta': 0.3203346608008277}\n{'n_estimators': 80, 'max_depth': 7, 'colsample_bytree': 0.5764929525944595, 'eta': 0.23497115410695124}\n\n"

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=151, max_depth=9, colsample_bytree=0.8012678112481448, eta=0.3203346608008277)
#RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=80, max_depth=7, colsample_bytree=0.5764929525944595, eta=0.23497115410695124)

In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}


_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=725769 / RMSE=11.00796521395197 / MAE=6.380813513951777 / MeanReal: 30.31119851337666 / MeanPred: 30.311437606811523
              Resid=-173.5402372649359 / AbsResidMean=6.380813513951777 / ResidMean=-0.0002391122206496748 / StdResidMean=1.5167312480079757e-05
Test scores: N=10 / RMSE=4.997775643346764 / MAE=4.544715225219727 / MeanReal: 30.019 / MeanPred: 26.8787784576416
             Resid=31.402200164794923 / AbsResidMean=4.544715225219727 / ResidMean=3.1402200164794922 / StdResidMean=0.38880303020079066
3:13:09.101057
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=725830 / RMSE=11.12728179341268 / MAE=6.436804876415253 / MeanReal: 30.311285264944775 / MeanPred: 30.311670303344727
              Resid=-278.52966626119223 / AbsResidMean=6.436804876415253 / ResidMean=-0.00038373953441194777 / StdResidMean=1.53304241858228

In [ ]:
pd.DataFrame(model_scores)

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,725769,11.007965,6.380814,30.311199,30.311199,-173.540237,6.380814,-0.000239,0.000015,10,4.997776,4.544715,30.019000,26.878778,31.402200,4.544715,3.140220,0.388803
1,2023-06-01,10,725830,11.127282,6.436805,30.311285,30.311285,-278.529666,6.436805,-0.000384,0.000015,10,8.432302,4.987465,32.026000,29.588110,24.378914,4.987465,2.437891,0.807220
2,2023-06-01,16,725912,11.256320,6.478026,30.311493,30.311493,-351.157361,6.478026,-0.000484,0.000016,9,4.461761,4.118864,29.983333,26.120111,34.768997,4.118864,3.863222,0.248025
3,2023-06-01,22,725965,10.972408,6.347965,30.311627,30.311627,-306.282584,6.347965,-0.000422,0.000015,10,12.957748,8.049310,34.290000,33.336910,9.530886,8.049310,0.953089,1.292265
4,2023-06-01,25,726005,11.136012,6.404803,30.312068,30.312068,-108.265234,6.404803,-0.000149,0.000015,10,12.616462,8.047855,33.840000,32.315666,15.243325,8.047855,1.524332,1.252404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,740870,11.163282,6.427088,30.574956,30.574956,-0.958312,6.427088,-0.000001,0.000015,16,4.292188,3.719936,46.303750,42.583813,59.518982,3.719936,3.719936,0.133828
131,2023-06-29,31,740991,11.118288,6.389766,30.578057,30.578057,-135.749520,6.389766,-0.000183,0.000015,20,11.349718,7.641139,49.814500,45.822731,79.835385,7.641139,3.991769,0.531229
132,2023-06-29,37,741105,11.112636,6.407710,30.580870,30.580870,-454.632824,6.407710,-0.000613,0.000015,13,1.990409,1.649279,43.133077,42.762718,4.814743,1.649279,0.370365,0.150434
133,2023-06-29,40,741136,11.121927,6.393238,30.581195,30.581195,-237.929211,6.393238,-0.000321,0.000015,11,5.340068,4.472909,40.043636,36.320705,40.952231,4.472909,3.722930,0.348030


In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID MODEL OPT. - SP (scores MAPE).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,725769,11.007965,6.380814,30.311199,30.311199,-173.540237,6.380814,-0.000239,0.000015,10,4.997776,4.544715,30.019000,26.878778,31.402200,4.544715,3.140220,0.388803
1,2023-06-01,10,725830,11.127282,6.436805,30.311285,30.311285,-278.529666,6.436805,-0.000384,0.000015,10,8.432302,4.987465,32.026000,29.588110,24.378914,4.987465,2.437891,0.807220
2,2023-06-01,16,725912,11.256320,6.478026,30.311493,30.311493,-351.157361,6.478026,-0.000484,0.000016,9,4.461761,4.118864,29.983333,26.120111,34.768997,4.118864,3.863222,0.248025
3,2023-06-01,22,725965,10.972408,6.347965,30.311627,30.311627,-306.282584,6.347965,-0.000422,0.000015,10,12.957748,8.049310,34.290000,33.336910,9.530886,8.049310,0.953089,1.292265
4,2023-06-01,25,726005,11.136012,6.404803,30.312068,30.312068,-108.265234,6.404803,-0.000149,0.000015,10,12.616462,8.047855,33.840000,32.315666,15.243325,8.047855,1.524332,1.252404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,740870,11.163282,6.427088,30.574956,30.574956,-0.958312,6.427088,-0.000001,0.000015,16,4.292188,3.719936,46.303750,42.583813,59.518982,3.719936,3.719936,0.133828
131,2023-06-29,31,740991,11.118288,6.389766,30.578057,30.578057,-135.749520,6.389766,-0.000183,0.000015,20,11.349718,7.641139,49.814500,45.822731,79.835385,7.641139,3.991769,0.531229
132,2023-06-29,37,741105,11.112636,6.407710,30.580870,30.580870,-454.632824,6.407710,-0.000613,0.000015,13,1.990409,1.649279,43.133077,42.762718,4.814743,1.649279,0.370365,0.150434
133,2023-06-29,40,741136,11.121927,6.393238,30.581195,30.581195,-237.929211,6.393238,-0.000321,0.000015,11,5.340068,4.472909,40.043636,36.320705,40.952231,4.472909,3.722930,0.348030


In [ ]:
model_scores_df['rmse_test'].mean()

9.661689875981512

In [ ]:
model_scores_df['meanreal_test'].mean()

41.86638982078944

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID MODEL OPT. - SP (feat.importances).csv'
#model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.254414,0.258679,0.248824,0.242066,0.254973,0.257536,0.249316,0.255785,0.245081,0.263170,0.249544,0.252015,0.263144,0.252538,0.257788,0.265755
1,BMU Fuel Type_label,0.173433,0.168668,0.168831,0.178096,0.182094,0.189267,0.177944,0.182774,0.179137,0.173641,0.159558,0.171999,0.172739,0.167774,0.160408,0.168561
2,BMU Type_label,0.076637,0.062321,0.059896,0.073401,0.068626,0.066780,0.065364,0.070965,0.067646,0.078960,0.089345,0.086466,0.084821,0.098229,0.088849,0.087883
3,Z1,0.047254,0.046077,0.049844,0.052210,0.045598,0.050131,0.048080,0.050410,0.046509,0.049732,0.043131,0.039876,0.051138,0.047729,0.040196,0.048151
4,Z6,0.044066,0.056149,0.051983,0.047375,0.048351,0.055007,0.050557,0.050050,0.054189,0.046121,0.034000,0.032219,0.033809,0.037467,0.029863,0.033845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Z9,0.000898,0.001058,0.000702,0.001016,0.000758,0.001199,0.000690,0.001444,0.000929,0.000746,0.000851,0.000836,0.000819,0.000809,0.001048,0.000565
69,sp6_12_totalvolume,0.000871,0.001407,0.000846,0.000877,0.000827,0.000834,0.000715,0.000703,0.000705,0.000746,0.000902,0.000964,0.000958,0.000894,0.000746,0.000936
70,Z15,0.000663,0.000602,0.000566,0.000797,0.000594,0.000766,0.001574,0.000916,0.001210,0.000508,0.000419,0.000508,0.000380,0.000226,0.000586,0.000294
71,FPN Flag_label,0.000651,0.000713,0.000760,0.000831,0.000875,0.000277,0.001114,0.000323,0.000979,0.000761,0.000568,0.000379,0.000451,0.000656,0.000411,0.000670


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.254414,0.258679,0.248824,0.242066,0.254973,0.257536,0.249316,0.255785,0.245081,0.263170,0.249544,0.252015,0.263144,0.252538,0.257788,0.265755
1,BMU Fuel Type_label,0.173433,0.168668,0.168831,0.178096,0.182094,0.189267,0.177944,0.182774,0.179137,0.173641,0.159558,0.171999,0.172739,0.167774,0.160408,0.168561
2,BMU Type_label,0.076637,0.062321,0.059896,0.073401,0.068626,0.066780,0.065364,0.070965,0.067646,0.078960,0.089345,0.086466,0.084821,0.098229,0.088849,0.087883
3,Z1,0.047254,0.046077,0.049844,0.052210,0.045598,0.050131,0.048080,0.050410,0.046509,0.049732,0.043131,0.039876,0.051138,0.047729,0.040196,0.048151
4,Z6,0.044066,0.056149,0.051983,0.047375,0.048351,0.055007,0.050557,0.050050,0.054189,0.046121,0.034000,0.032219,0.033809,0.037467,0.029863,0.033845
5,Transmission Loss Factor,0.031693,0.032337,0.031754,0.032579,0.028072,0.038675,0.027209,0.027558,0.028228,0.028166,0.029071,0.033059,0.039520,0.034118,0.031023,0.034035
6,Weekend,0.029426,0.052185,0.063771,0.054202,0.035702,0.039527,0.000842,0.025322,0.030372,0.031853,0.036349,0.029621,0.005498,0.000878,0.030613,0.004661
7,LOC Center LONG,0.023319,0.034959,0.026959,0.032583,0.032541,0.026742,0.035485,0.019117,0.015264,0.017107,0.015230,0.018359,0.013915,0.018016,0.027185,0.016315
8,LOC Center LAT,0.021141,0.027107,0.021742,0.023688,0.024350,0.023550,0.017053,0.018485,0.016830,0.016477,0.020614,0.021266,0.020610,0.019322,0.021872,0.024146
9,LOC LONG,0.019059,0.012219,0.014159,0.015606,0.019718,0.012036,0.015887,0.023883,0.016335,0.022515,0.020913,0.022119,0.022801,0.022520,0.022347,0.022826


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
53,BZONE_label,0.001942,0.002266,0.001986,0.002070,0.001675,0.001916,0.002047,0.002161,0.002021,0.002137,0.001892,0.001549,0.001947,0.002316,0.001359,0.001784
54,DAImbalance,0.001819,0.000685,0.003166,0.000715,0.001281,0.001080,0.000953,0.002032,0.001178,0.001073,0.001539,0.001629,0.003636,0.002557,0.003479,0.002278
55,marketIndexVolume,0.001803,0.000483,0.000841,0.003218,0.000936,0.000801,0.002128,0.001368,0.000755,0.001445,0.001475,0.001188,0.003038,0.004086,0.004397,0.000882
56,settlementPeriod_ordin,0.001627,0.000727,0.000743,0.000836,0.000741,0.000876,0.000774,0.001635,0.000969,0.001262,0.000945,0.003640,0.001858,0.003863,0.002156,0.003372
57,sp3_6_totalvolume,0.001592,0.001217,0.001123,0.001174,0.002884,0.001416,0.001174,0.001322,0.001852,0.001402,0.002047,0.001463,0.001445,0.002003,0.001798,0.001565
58,Z13,0.001448,0.001498,0.001196,0.001362,0.001506,0.001556,0.001284,0.001316,0.000984,0.001480,0.001692,0.001640,0.001053,0.001746,0.001875,0.001538
59,Holiday,0.001413,0.001749,0.001974,0.001330,0.002121,0.001439,0.000994,0.001498,0.001250,0.001717,0.001604,0.001165,0.001650,0.000895,0.000822,0.000991
60,DATF_Demand,0.001410,0.000994,0.000878,0.000902,0.000974,0.000842,0.001565,0.000916,0.001155,0.000769,0.001240,0.005550,0.001003,0.001250,0.001422,0.001689
61,SystemTotalLoad_Consumption,0.001347,0.001190,0.001299,0.001309,0.001398,0.001371,0.002417,0.000957,0.001008,0.001056,0.001187,0.001047,0.001189,0.002675,0.000978,0.001120
62,WindOffshore_Generation,0.001237,0.001075,0.001063,0.001094,0.001188,0.001131,0.001923,0.001182,0.001097,0.001163,0.001583,0.001181,0.001154,0.001186,0.001112,0.001419


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### XGBOOST (SP PAVG. PRICE MODEL) - Modelo BID (Predicción por SP)

In [186]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [187]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

229670
5
229700
6
229743
6
229782
7
229808
7
229846
7
229884
8
229908
10
229974
5
230277
5
230312
8
230343
4
230374
4
230389
4
230409
5
230442
11
230466
5
230509
2
230680
5
230705
6
230733
3
230748
1
230754
4
230782
7
230825
9
230847
8
230914
3
231130
7
231166
5
231206
5
231227
0
231227
4
231254
5
231291
3
231297
3
231340
8
231662
6
231683
2
231716
4
231750
5
231759
3
231785
3
231816
6
231834
4
231878
5
232090
1
232097
0
232122
3
232149
10
232165
2
232199
4
232218
9
232232
9
232269
4
232607
9
232639
7
232682
12
232729
7
232749
8
232785
4
232815
5
232835
7
232898
5
233161
2
233172
1
233217
7
233272
9
233293
2
233312
8
233347
3
233350
1
233393
6
233675
2
233699
3
233731
5
233768
6
233789
6
233830
7
233871
4
233886
5
233925
5
234212
4
234244
5
234294
8
234331
5
234346
4
234374
4
234402
6
234428
11
234518
9
234815
4
234836
7
234879
7
234921
3
234932
7
234969
4
234995
5
235021
5
235063
9
235357
8
235398
7
235441
5
235469
7
235487
5
235519
1
235542
13
235570
5
235614
7
235872
7
235904
4
2359

In [188]:
unselect_vars = ['recordtype_label', 'Year_ordin']

In [189]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

len(select_vars)

40

In [190]:
'''
{'n_estimators': 55, 'max_depth': 2, 'colsample_bytree': 0.06685323090676709, 'eta': 0.2893124969719224}
'''

"\n{'n_estimators': 55, 'max_depth': 2, 'colsample_bytree': 0.06685323090676709, 'eta': 0.2893124969719224}\n"

In [191]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=55, max_depth=2, colsample_bytree=0.06685323090676709, eta=0.2893124969719224)

In [192]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}


_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=229670 / RMSE=73.86835228724483 / MAE=27.140193109536316 / MeanReal: 73.17324911328387 / MeanPred: 73.17324829101562
              Resid=0.8935487756672167 / AbsResidMean=27.140193109536316 / ResidMean=3.890576814122345e-06 / StdResidMean=0.0003216282156452508
Test scores: N=5 / RMSE=9.011282213700046 / MAE=7.9358058287963305 / MeanReal: 30.074272473083436 / MeanPred: 22.13846778869629
             Resid=39.67902914398165 / AbsResidMean=7.9358058287963305 / ResidMean=7.9358058287963305 / StdResidMean=0.8538429125451354
0:03:56.831288
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=229700 / RMSE=73.8638780716725 / MAE=27.13869670766287 / MeanReal: 73.16778496979659 / MeanPred: 73.16777801513672
              Resid=0.6345672975111043 / AbsResidMean=27.13869670766287 / ResidMean=2.7625916352669694e-06 / StdResidMean=0.00032156

In [193]:
pd.DataFrame(model_scores).sort_values(by=['rmse_test'], ascending=False).head(20)

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
109,2023-06-25,10,235904,74.598060,26.835283,72.656736,72.656736,1.552897,26.835283,0.000007,0.000316,4,104.317341,82.130300,-85.250531,-22.145018,-252.422057,82.130300,-63.105514,20.766261
110,2023-06-25,16,235938,74.601396,26.844671,72.640875,72.640875,1.519427,26.844671,0.000006,0.000316,3,103.012031,82.199234,-105.365296,-23.166061,-246.597701,82.199234,-82.199234,20.695583
108,2023-06-25,4,235872,74.563596,26.772238,72.674834,72.674834,-2.090720,26.772238,-0.000009,0.000316,7,90.643104,57.925237,-51.233803,-0.473675,-355.320895,57.925237,-50.760128,10.728164
112,2023-06-25,25,235964,74.600967,26.847080,72.631260,72.631260,1.463016,26.847080,0.000006,0.000316,3,79.675066,68.804263,36.732410,-32.071854,206.412789,68.804263,68.804263,13.391911
53,2023-06-11,48,232269,75.138198,27.160018,72.825193,72.825193,12.085751,27.160018,0.000052,0.000323,4,67.194501,64.485618,53.493750,117.979370,-257.942473,64.485618,-64.485618,4.721666
45,2023-06-11,4,232090,73.437092,26.960681,72.788187,72.788187,141.122352,26.960681,0.000608,0.000316,1,62.702736,62.702736,21.014805,-41.687931,62.702736,62.702736,62.702736,0.000000
111,2023-06-25,22,235956,74.601197,26.846408,72.633219,72.633219,1.322345,26.846408,0.000006,0.000316,3,59.874685,46.194865,12.900073,-33.294792,138.584595,46.194865,46.194865,12.697385
26,2023-06-05,48,230914,73.641019,27.129843,72.957383,72.957383,142.348014,27.129843,0.000616,0.000319,3,52.665912,30.777319,59.581990,28.825251,92.270216,30.777319,30.756739,14.250619
37,2023-06-09,10,231683,73.576484,27.098561,72.865966,72.865966,53.476296,27.098561,0.000231,0.000318,2,41.708719,41.677137,49.854441,8.177304,83.354274,41.677137,41.677137,0.811397
113,2023-06-25,31,235991,74.601664,26.849849,72.629201,72.629201,1.403712,26.849849,0.000006,0.000316,2,39.410816,37.530294,-0.283554,-37.813847,75.060588,37.530294,37.530294,6.014345


In [194]:
model_scores_df = pd.DataFrame(model_scores)

In [195]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID SP PAVG. PRICE MODEL OPT. - SP (scores MAPE).csv'
model_scores_df.to_csv(file_path, index=False)

In [196]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,229670,73.868352,27.140193,73.173249,73.173249,0.893549,27.140193,0.000004,0.000322,5,9.011282,7.935806,30.074272,22.138468,39.679029,7.935806,7.935806,0.853843
1,2023-06-01,10,229700,73.863878,27.138697,73.167785,73.167785,0.634567,27.138697,0.000003,0.000322,6,23.619800,22.809780,32.191245,9.381466,136.858678,22.809780,22.809780,1.022102
2,2023-06-01,16,229743,73.858186,27.137925,73.160284,73.160284,1.140961,27.137925,0.000005,0.000321,6,14.349966,13.065280,30.040695,17.374048,75.999884,13.065280,12.666647,1.123951
3,2023-06-01,22,229782,73.852707,27.136366,73.153514,73.153514,1.183928,27.136366,0.000005,0.000321,7,10.733756,8.481880,29.932998,28.390854,10.795013,8.481880,1.542145,1.517485
4,2023-06-01,25,229808,73.849109,27.135165,73.149549,73.149549,1.463546,27.135165,0.000006,0.000321,7,14.749487,10.377737,29.423439,23.621286,40.615077,10.377737,5.802154,1.937189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,237099,74.490246,26.888219,72.447233,72.447233,1.679740,26.888219,0.000007,0.000314,8,11.580501,7.421622,53.973159,54.912834,-7.517405,7.421622,-0.939676,1.442789
131,2023-06-29,31,237161,74.481078,26.884089,72.442964,72.442964,1.596234,26.884089,0.000007,0.000314,9,27.532739,21.293028,57.624310,41.658634,143.691081,21.293028,15.965676,2.492331
132,2023-06-29,37,237216,74.473113,26.881636,72.439203,72.439203,1.810977,26.881636,0.000008,0.000314,7,15.941256,14.915366,53.403294,44.327709,63.529085,14.915366,9.075584,1.872232
133,2023-06-29,40,237231,74.470948,26.881079,72.437787,72.437787,1.285373,26.881079,0.000005,0.000314,6,16.236559,13.955520,52.449252,47.960144,26.934656,13.955520,4.489109,2.600608


In [197]:
model_scores_df['rmse_test'].mean()

21.932010881167873

In [207]:
model_scores_df['mae_test'].mean()

18.157109536795733

In [198]:
model_scores_df['meanreal_test'].mean()

45.28154068887807

In [199]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [200]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost BID SP PAVG. PRICE MODEL OPT. - SP (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [201]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.158691,0.158440,0.158981,0.159032,0.158562,0.159183,0.157737,0.156912,0.157347,0.157579,0.158608,0.159168,0.159291,0.159962,0.159695,0.159870
1,Coal_price,0.134116,0.134891,0.135189,0.134904,0.134417,0.135287,0.133944,0.133090,0.133012,0.132791,0.133615,0.133822,0.133624,0.133733,0.134648,0.134776
2,DAMargin,0.121318,0.120908,0.122211,0.122294,0.121787,0.122322,0.122491,0.121534,0.121027,0.120869,0.121439,0.121196,0.120645,0.120671,0.120483,0.119888
3,sp6_12_pavgprice,0.098127,0.097974,0.098405,0.098379,0.097965,0.098516,0.097729,0.097297,0.097233,0.097336,0.097988,0.098338,0.098381,0.098509,0.098901,0.098957
4,Month_SIN,0.079448,0.079297,0.079626,0.079688,0.079506,0.079950,0.079049,0.078648,0.078583,0.078625,0.079464,0.079824,0.079854,0.079830,0.079833,0.079937
5,BrentOil_price,0.055219,0.055759,0.055707,0.055519,0.055206,0.055430,0.054895,0.054595,0.054668,0.054760,0.055253,0.055504,0.055546,0.055376,0.055012,0.055052
6,WindOffshore_Generation,0.039805,0.040367,0.040290,0.039998,0.039751,0.040003,0.039793,0.039504,0.039580,0.039562,0.039683,0.039789,0.039735,0.039907,0.039618,0.039498
7,WindOnshore_Generation,0.034143,0.032135,0.032250,0.032163,0.032135,0.032215,0.039419,0.039308,0.039394,0.039483,0.032172,0.032370,0.032382,0.032307,0.032183,0.032230
8,DATF_Demand,0.033818,0.033246,0.033621,0.033778,0.033796,0.034051,0.033790,0.033573,0.033644,0.033555,0.033758,0.033880,0.033838,0.034082,0.034276,0.034377
9,DAImbalance,0.028200,0.028265,0.028447,0.028354,0.028230,0.028274,0.028060,0.027880,0.027924,0.027975,0.028140,0.028244,0.028235,0.028352,0.028328,0.028286


In [202]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.158691,0.158440,0.158981,0.159032,0.158562,0.159183,0.157737,0.156912,0.157347,0.157579,0.158608,0.159168,0.159291,0.159962,0.159695,0.159870
1,Coal_price,0.134116,0.134891,0.135189,0.134904,0.134417,0.135287,0.133944,0.133090,0.133012,0.132791,0.133615,0.133822,0.133624,0.133733,0.134648,0.134776
2,DAMargin,0.121318,0.120908,0.122211,0.122294,0.121787,0.122322,0.122491,0.121534,0.121027,0.120869,0.121439,0.121196,0.120645,0.120671,0.120483,0.119888
3,sp6_12_pavgprice,0.098127,0.097974,0.098405,0.098379,0.097965,0.098516,0.097729,0.097297,0.097233,0.097336,0.097988,0.098338,0.098381,0.098509,0.098901,0.098957
4,Month_SIN,0.079448,0.079297,0.079626,0.079688,0.079506,0.079950,0.079049,0.078648,0.078583,0.078625,0.079464,0.079824,0.079854,0.079830,0.079833,0.079937
5,BrentOil_price,0.055219,0.055759,0.055707,0.055519,0.055206,0.055430,0.054895,0.054595,0.054668,0.054760,0.055253,0.055504,0.055546,0.055376,0.055012,0.055052
6,WindOffshore_Generation,0.039805,0.040367,0.040290,0.039998,0.039751,0.040003,0.039793,0.039504,0.039580,0.039562,0.039683,0.039789,0.039735,0.039907,0.039618,0.039498
7,WindOnshore_Generation,0.034143,0.032135,0.032250,0.032163,0.032135,0.032215,0.039419,0.039308,0.039394,0.039483,0.032172,0.032370,0.032382,0.032307,0.032183,0.032230
8,DATF_Demand,0.033818,0.033246,0.033621,0.033778,0.033796,0.034051,0.033790,0.033573,0.033644,0.033555,0.033758,0.033880,0.033838,0.034082,0.034276,0.034377
9,DAImbalance,0.028200,0.028265,0.028447,0.028354,0.028230,0.028274,0.028060,0.027880,0.027924,0.027975,0.028140,0.028244,0.028235,0.028352,0.028328,0.028286


In [203]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.158691,0.158440,0.158981,0.159032,0.158562,0.159183,0.157737,0.156912,0.157347,0.157579,0.158608,0.159168,0.159291,0.159962,0.159695,0.159870
1,Coal_price,0.134116,0.134891,0.135189,0.134904,0.134417,0.135287,0.133944,0.133090,0.133012,0.132791,0.133615,0.133822,0.133624,0.133733,0.134648,0.134776
2,DAMargin,0.121318,0.120908,0.122211,0.122294,0.121787,0.122322,0.122491,0.121534,0.121027,0.120869,0.121439,0.121196,0.120645,0.120671,0.120483,0.119888
3,sp6_12_pavgprice,0.098127,0.097974,0.098405,0.098379,0.097965,0.098516,0.097729,0.097297,0.097233,0.097336,0.097988,0.098338,0.098381,0.098509,0.098901,0.098957
4,Month_SIN,0.079448,0.079297,0.079626,0.079688,0.079506,0.079950,0.079049,0.078648,0.078583,0.078625,0.079464,0.079824,0.079854,0.079830,0.079833,0.079937
5,BrentOil_price,0.055219,0.055759,0.055707,0.055519,0.055206,0.055430,0.054895,0.054595,0.054668,0.054760,0.055253,0.055504,0.055546,0.055376,0.055012,0.055052
6,WindOffshore_Generation,0.039805,0.040367,0.040290,0.039998,0.039751,0.040003,0.039793,0.039504,0.039580,0.039562,0.039683,0.039789,0.039735,0.039907,0.039618,0.039498
7,WindOnshore_Generation,0.034143,0.032135,0.032250,0.032163,0.032135,0.032215,0.039419,0.039308,0.039394,0.039483,0.032172,0.032370,0.032382,0.032307,0.032183,0.032230
8,DATF_Demand,0.033818,0.033246,0.033621,0.033778,0.033796,0.034051,0.033790,0.033573,0.033644,0.033555,0.033758,0.033880,0.033838,0.034082,0.034276,0.034377
9,DAImbalance,0.028200,0.028265,0.028447,0.028354,0.028230,0.028274,0.028060,0.027880,0.027924,0.027975,0.028140,0.028244,0.028235,0.028352,0.028328,0.028286


In [204]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
20,settlementDate_ordin,0.005476,0.004781,0.004873,0.004853,0.004891,0.004954,0.004993,0.005872,0.005944,0.005975,0.006146,0.006175,0.006190,0.006545,0.004945,0.005003
21,DAID_spnDemand,0.005257,0.004534,0.004550,0.004636,0.005517,0.004717,0.005442,0.005407,0.005404,0.005410,0.005466,0.005448,0.005477,0.005502,0.005656,0.005685
22,LOC Center LAT,0.005088,0.005097,0.005103,0.005089,0.005068,0.005591,0.005037,0.005000,0.004589,0.004584,0.005016,0.005029,0.005036,0.005044,0.005508,0.005526
23,DAIG_spnGeneration,0.004356,0.004151,0.004017,0.004813,0.004544,0.004258,0.004053,0.003976,0.004046,0.004054,0.004507,0.004460,0.004466,0.004328,0.004821,0.004840
24,settlementPeriod_ordin,0.003685,0.003608,0.003705,0.003709,0.003943,0.003476,0.003663,0.003682,0.003758,0.003708,0.003922,0.004192,0.004179,0.003813,0.002958,0.002951
25,GenerationForecast_Production,0.002626,0.002722,0.002662,0.002636,0.002543,0.002690,0.002551,0.002499,0.002584,0.002593,0.002555,0.002520,0.002533,0.002881,0.002706,0.002716
26,Weekday_SIN,0.002269,0.002261,0.002344,0.002461,0.002328,0.002340,0.002233,0.002255,0.002181,0.002174,0.002262,0.002222,0.002232,0.002285,0.002273,0.002184
27,sp6_12_meanvolume,0.002088,0.002084,0.002071,0.002044,0.002107,0.002024,0.002131,0.002106,0.002102,0.002102,0.002113,0.002111,0.002105,0.002091,0.002073,0.002063
28,Day_SIN,0.002039,0.002002,0.001986,0.001973,0.002071,0.002066,0.002044,0.002030,0.002109,0.002113,0.002042,0.002087,0.002083,0.002056,0.001978,0.001947
29,settPeriod_COS,0.001779,0.001574,0.001470,0.001483,0.001477,0.001601,0.002036,0.002024,0.002030,0.002011,0.002015,0.001858,0.001514,0.001476,0.002063,0.002059


In [205]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [206]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

# Varios

### XGBOOST (BID/OFFER MODEL) - Modelo BID (Predicción por SP)

In [259]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}

test_days = []
test_sps = []

meanreal_trains = []
meanreal_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'],:]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'],:]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    meanreal_train = y_train.mean()

    size_test = y_test.shape[0]
    if size_test > 0:
      meanreal_test = y_test.mean()
    else:
      meanreal_test = np.nan

    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)


model_scores = {'test_day':test_days,'test_sp':test_sps,
                'meanreal_train':meanreal_trains,
                'meanreal_test':meanreal_tests}


In [261]:
model_scores_df = pd.DataFrame(model_scores)

In [262]:
model_scores_df['meanreal_test'].mean()

41.86638982078944

### XGBOOST (SP PAVG. PRICE MODEL) - Modelo BID (Predicción por SP)

In [263]:
recordtype = 'BID'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
recordtype = 'BID'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}

test_days = []
test_sps = []

meanreal_trains = []
meanreal_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'],:]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],:]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    meanreal_train = y_train.mean()

    size_test = y_test.shape[0]
    if size_test > 0:
      meanreal_test = y_test.mean()
    else:
      meanreal_test = np.nan

    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)


model_scores = {'test_day':test_days,'test_sp':test_sps,
                'meanreal_train':meanreal_trains,
                'meanreal_test':meanreal_tests}


In [265]:
model_scores_df = pd.DataFrame(model_scores)

In [266]:
model_scores_df['meanreal_test'].mean()

45.28154068887807